## MAPS analysis

this method was refered to Lee et.al,Integrative analysis reveals RNA G-quadruplexes in UTRs are selectively constrained and enriched for functional associations,Nat Commun,2020.

#### data prepared 
the methylation_hg38.RData and mutation_rate_methylation_bins.txt were download from https://zenodo.org/record/4536050/files/methylation_hg38.RData?download=1, this data was used to control methylation levels(Lee, et al. Disrupting upstream translation in mRNAs is associated with human disease. *Nat Communications* 2021)



In [ ]:
## maps_noG4_g4_data_get.py
import pandas as pd
import pyranges as pr
import os,sys
wdpath = "/disk2/G4_MS/frequency/MAPS/data"
gruns = pr.PyRanges(pd.read_table("/disk2/G4_MS/reference/genomic_Gruns.bed", header=None, names=["Chromosome", "Start", "End" ,"Strand", "grun"]))
promoter = pr.PyRanges(pd.read_table("/disk2/G4_MS/promoter_select/promoter_info/promoter.annotion.tsv"))
epromoter = pr.PyRanges(pd.read_table("/disk2/G4_MS/promoter_select/promoter_info/promoter_info.annotion.H3K4me3.tsv"))
promoter_pG4 = pr.PyRanges(pd.read_table("/disk2/G4_MS/promoter_select/promoter_info/promoter_pG4.info.tsv").loc[:,["Chromosome", "Start","End","Strand","Name","gene_id","transcript_id","Score", "Strand_b"]])
promoter_eG4 = pr.PyRanges(pd.read_table("/disk2/G4_MS/promoter_select/promoter_info/promoter_pG4.info.H3K4me3.tsv").loc[:,["Chromosome", "Start","End","Strand","Name", "gene_id","transcript_id","Score", "Strand_b"]])
all_pG4 = pr.read_bed("/disk2/G4_MS/G4_identify/Quadron/hs38d2.quadron.bed")
def gruns_select(gruns, all_pG4, promoter, promoter_pG4, eG4=False):
    g4 = "pG4"
    if eG4:
        g4 = "eG4"
    promoter_noG4 = promoter.subtract(all_pG4,strandedness=False)
    promoter_pG4_gruns = promoter_pG4.intersect(gruns, strandedness="same").drop_duplicate_positions().as_df().loc[:,["Chromosome", "Start","End","transcript_id","gene_id","Strand"]]
    promoter_pG4_gruns.to_csv(os.path.join(wdpath,"promoter_1000_"+g4+".gruns.bed"),index=False, header=False, sep="\t")
    
    promoter_nonG4_gruns = promoter_noG4.intersect(gruns,strandedness=False).drop_duplicate_positions().as_df().loc[:,["Chromosome", "Start","End","transcript_id","gene_id","Strand"]]
    promoter_nonG4_gruns.to_csv(os.path.join(wdpath, "protein_promoter_1000_noG4."+ g4 + ".gruns.bed"), index=False, header=False, sep="\t")
    
    pre_promoter_pG4 = promoter_pG4.drop_duplicate_positions().as_df().loc[:,["Chromosome", "Start","End","Name", "Strand","transcript_id","gene_id","Strand_b"]]
    if eG4:
        pre_promoter_pG4.to_csv(os.path.join(wdpath, "promoter_pG4.info.H3K4me3.tsv"),index=False, header=False, sep="\t")
    else:
        pre_promoter_pG4.to_csv(os.path.join(wdpath, "promoter_"+g4+".info.tsv"),index=False, header=False, sep="\t")
    
    promoter_stable = promoter_pG4[promoter_pG4.Score >19]
    promoter_unstable = promoter_pG4[promoter_pG4.Score <= 19]
    
    promoter_stable_gruns = promoter_stable.intersect(gruns,strandedness="same").drop_duplicate_positions().as_df().loc[:,["Chromosome", "Start","End","transcript_id","gene_id","Strand"]]
    promoter_stable_gruns.to_csv(os.path.join(wdpath,"promoter_1000_stable_"+g4+".gruns.bed"),index=False, header=False, sep="\t")
    
    promoter_unstable_gruns = promoter_unstable.intersect(gruns,strandedness="same").drop_duplicate_positions().as_df().loc[:,["Chromosome", "Start","End","transcript_id","gene_id","Strand"]]
    promoter_unstable_gruns.to_csv(os.path.join(wdpath,"promoter_1000_unstable_"+g4+".gruns.bed"),index=False, header=False, sep="\t")
    
    promoter = promoter.drop_duplicate_positions().as_df().loc[:,["Chromosome","Start", "End","Promoter_id", "gene_id", "Strand","transcript_id"]]
    if eG4:
        promoter.to_csv(os.path.join(wdpath, "promoter_info.annotion.H3K4me3.tsv"),index=False, header=False, sep="\t")
    else:
        promoter.to_csv(os.path.join(wdpath, "promoter.annotion.tsv"),index=False, header=False, sep="\t")

gruns_select(gruns, all_pG4, promoter, promoter_pG4)
gruns_select(gruns, all_pG4, epromoter, promoter_eG4, eG4=True)


In [ ]:
# maps_synonynous.py
import os,sys,gzip
import pandas as pd
import pybedtools
wdpath = "/disk2/G4_MS/frequency/MAPS/data"
# refpath = "/usr/refence/hg38/hg38.fa"
with gzip.open(os.path.join(wdpath,"cds.gnomad.genomes.v3.1.sites.pass_snps.tsv.gz"),'rt')as fr,open(os.path.join(wdpath,"gnomad.genomes.v3.1.sites.pass_snps.synonymous.tsv"),'w')as fw, open(os.path.join(wdpath,"gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv"),'w')as fw2:
    for line in fr:
        uline = line.strip().split("\t")
        chromsome,start,end,ref,alter,ac,an,af,info=uline
        if "synonymous_variant" in info and "missense_variant" not in info:
            fw.write("\t".join([chromsome,start,ref,alter,an,ac,af,"snv",'.',info])+"\n")
        if "missense_variant" in info:
            fw2.write("\t".join([chromsome,start,ref,alter,an,ac,af,"snv",'.',info])+"\n")

In [ ]:
# snp_trans.py 
import sys,os
import gzip
wdpath = "/disk2/G4_MS/frequency/MAPS/data"
with gzip.open(os.path.join(wdpath,"promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv.gz"),'rt')as fr,open(os.path.join(wdpath,"promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv"),'w')as fw:
    #fw.write("chromosome\tstart\tref\talt\tan\tac\taf\n")
    for line in fr:
        uline = line.strip().split("\t")
        chromsome,start,end,ref,alter,ac,an,af,info=uline
        fw.write("\t".join([chromsome,start,ref,alter,an,ac,af])+"\n")
        
with gzip.open(os.path.join(wdpath,"active_promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv.gz"),'rt')as fr,open(os.path.join(wdpath,"active_promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv"),'w')as fw:
    #fw.write("chromosome\tstart\tref\talt\tan\tac\taf\n")
    for line in fr:
        uline = line.strip().split("\t")
        chromsome,start,end,ref,alter,ac,an,af,info=uline
        fw.write("\t".join([chromsome,start,ref,alter,an,ac,af])+"\n")        

with gzip.open("/disk2/G4_MS/reference/gnomad.genomes.v3.1.sites.pass_snps.tsv.gz",'rt')as fr,open(os.path.join(wdpath,"gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv"),'w')as fw:
    for line in fr:
        uline = line.strip().split("\t")
        chromsome,start,end,ref,alter,ac,an,af,info=uline
        if "HIGH" in info:
            fw.write("\t".join([chromsome,start,ref,alter,an,ac,af,"snv",'.',info])+"\n")

In [ ]:
# Title     : TODO
# Objective : TODO
# Created by: guang
# Created on: 2021/9/30
# Filname maps.model.R
library(dplyr)
library(ggplot2)
load("/disk2/G4_MS/frequency/MAPS/data/methylation_hg38.RData")

make_ps_model <- function(cds_vcf, mut, AC_thresh = 1) {
  singletons <- cds_vcf %>%
    filter(AC == AC_thresh)
  singletons_context <- table(singletons$mut)
  context_n <- table(cds_vcf$mut)
  prop_singletons <- data.frame('mutation' = names(singletons_context),
                                'singletons' = c(singletons_context))
  prop_n <- data.frame('mutation' = names(context_n),
                       'n' = c(context_n))
  prop_singletons <- left_join(prop_singletons, prop_n, by ='mutation')

  prop_singletons <- left_join(prop_singletons, mut, by = 'mutation')
  ps_lm <- lm(singletons/n ~ mu_snp, data = prop_singletons, weights = n)
  return(ps_lm)

}

make_ps2_model <- function(cds_vcf, mu, AC_thresh = 1) {
  variants <- split(cds_vcf, cds_vcf$group)
  ps_models <- lapply(variants, function(x) make_ps_model(x, mu, AC_thresh))
  return(ps_models)
}

annotate_methylation <- function(variants, mutation_table, methylation) {
  variants <- left_join(variants, methylation)
  # set missing methylation values to zero
  variants$methyl_bins[is.na(variants$methyl_bins)] <- 0
  fixed_context <- ifelse(variants$context %in% mutation_table$context,
                          variants$context,
                          chartr('ATGC','TACG',variants$context))
  fixed_mut <- ifelse(variants$context %in% mutation_table$context,
                      variants$mut_context,
                      chartr('ATGC','TACG',variants$mut_context))

  # amend mutation context for MAPS score calculation
  variants$mut <- paste0(fixed_context,':',
                         variants$methyl_bins,
                         '>',
                         fixed_mut)

  variants$mut <- ifelse(variants$mut %in% mutation_table$mutation,
                         variants$mut,
                         gsub('[1|2]',
                              '0',
                              variants$mut))

  variants$context <- fixed_context
  variants$mut_context <- fixed_mut

  # annotate variants for methylation adjustment
  var_adjust <- mutation_table %>%
    filter(methylation_level != 0) %>%
    select(context) %>% unlist()

  variants$group <- ifelse((variants$context %in% var_adjust),
                           ifelse((variants$REF == 'C' & variants$ALT == 'T') | (variants$REF == 'G' & variants$ALT == 'A'),
                                  'methyl_adjust',
                                  'no_adjust'),
                           'no_adjust')

  return(variants)

}
bed_dir <- "/disk2/G4_MS/frequency/MAPS/data"
setwd(bed_dir)
# load variants from gnomAD v3.1
vcf_syn <- data.table::fread("gnomad.genomes.v3.1.sites.pass_snps.synonymous.tsv")
colnames(vcf_syn) <- c('CHROM','POS','REF','ALT','AN','AC','AF','variant_type','lcr','vep')
seq <- data.table::fread('gnomad.genomes.v3.1.sites.pass_snps.synonymous.seq',
                        header = FALSE)
seq <- toupper(seq$V2)
vcf_syn$heptamer <- seq
# filter variants
vcf_syn <- vcf_syn %>%
    filter(lcr != 1, AN >= 121849, CHROM != 'chrX', CHROM != 'chrY',variant_type == 'snv')
vcf_syn$context <- substr(vcf_syn$heptamer, start = 3, stop = 5)
vcf_syn$mut_context <- paste0(substr(vcf_syn$context, 1, 1),
                      vcf_syn$ALT,
                      substr(vcf_syn$context, 3, 3))

# load mutation rate table
mut <- data.table::fread('mutation_rate_methylation_bins.txt')
mut <- mut %>%
  mutate(ref_context = paste0(context, ':', methylation_level),
         mut_context = paste0(substr(context, 1, 1),
                              alt,
                              substr(context, 3, 3)),
         mutation = paste0(ref_context, '>', mut_context))




# annotate methylation levels
vcf_syn <- annotate_methylation(vcf_syn, mut, combined_methylation_estimates)
vcf_syn$methyl_bins <- as.numeric(as.character(vcf_syn$methyl_bins))
# make MAPS model
ps_lm_genomes <- make_ps2_model(vcf_syn, mut, AC_thresh = 1)
lapply(ps_lm_genomes, function(x) summary(x))
if (!dir.exists('RData')) {
  dir.create('RData')
}
save(ps_lm_genomes, file = 'RData/ps_lm_genomes.RData')

In [ ]:
%%bash
cd /disk2/G4_MS/frequency/MAPS/data
python maps_noG4_g4_data_get.py
bedtools intersect -a /disk2/G4_MS/reference/gnomad.genomes.v3.1.sites.pass_snps.tsv.gz -b /disk2/G4_MS/reference/gencode38.cds_region.bed -wa -u|gzip > cds.gnomad.genomes.v3.1.sites.pass_snps.tsv.gz
python maps_synonynous.py
python snp_trans.py
bedtools sort -i gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv > gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv1
mv gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv1 gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv
awk -v OFS="\t" '{print$1,$2-4,$2+3}' gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv|bedtools getfasta -fi /usr/refence/hg38/hg38.fa -bed - -tab >gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.seq

bedtools sort -i gnomad.genomes.v3.1.sites.pass_snps.synonymous.tsv > gnomad.genomes.v3.1.sites.pass_snps.synonymous.tsv1
mv gnomad.genomes.v3.1.sites.pass_snps.synonymous.tsv1 gnomad.genomes.v3.1.sites.pass_snps.synonymous.tsv
awk -v OFS="\t" '{print$1,$2-4,$2+3}' gnomad.genomes.v3.1.sites.pass_snps.synonymous.tsv |bedtools getfasta -fi /usr/refence/hg38/hg38.fa -bed - -tab > gnomad.genomes.v3.1.sites.pass_snps.synonymous.seq

tail -n +2 /disk2/G4_MS/promoter_select/promoter_info/promoter.annotion.tsv|bedtools sort -i - |bedtools merge -i -|bedtools intersect -a /disk2/G4_MS/reference/gnomad.genomes.v3.1.sites.pass_snps.tsv.gz -b -|bedtools intersect -a - -b /disk2/G4_MS/reference/gencode38.cds_region.bed -v |bedtools sort -i -|gzip > promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv.gz
tail -n +2 /disk2/G4_MS/promoter_select/promoter_info/promoter_info.annotion.H3K4me3.tsv|bedtools sort -i - |bedtools merge -i -|bedtools intersect -a /disk2/G4_MS/reference/gnomad.genomes.v3.1.sites.pass_snps.tsv.gz -b -|bedtools intersect -a - -b /disk2/G4_MS/reference/gencode38.cds_region.bed -v |bedtools sort -i - |gzip > active_promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv.gz
python snp_trans.py
awk -v OFS="\t" '{print$1,$2-4,$2+3}' promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv|bedtools getfasta -fi /usr/refence/hg38/hg38.fa -bed - -tab > promoter_region_gnomad.genomes.v3.1.sites.pass_snps.seq
awk -v OFS="\t" '{print$1,$2-4,$2+3}' active_promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv|bedtools getfasta -fi /usr/refence/hg38/hg38.fa -bed - -tab > active_promoter_region_gnomad.genomes.v3.1.sites.pass_snps.seq

# bedtools sort -i gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv > gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv1
# mv gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv1 gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv
# awk -v OFS="\t" '{print$1,$2-4,$2+3}' gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv|bedtools getfasta -fi /usr/refence/hg38/hg38.fa -bed - -tab > gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv.seq

bedtools sort -i gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv > gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv1
mv gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv1 gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv
awk -v OFS="\t" '{print$1,$2-4,$2+3}' gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv|bedtools getfasta -fi /usr/refence/hg38/hg38.fa -bed - -tab > gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv.seq

Rscript maps.model.R

In [ ]:
# Title     : TODO
# Objective : TODO
# Created by: guangyue
# Created on: 2021/9/30
# h3k4me3_stable_unstable.data.get.R

library(GenomicRanges)
library(dplyr)


######################
## Helper functions ##
######################
granges_from_vcf <- function(vcf) {

  vcf$end <- vcf$POS
  grange <- makeGRangesFromDataFrame(vcf, keep.extra.columns = TRUE, ignore.strand = TRUE,
                                     seqnames.field = "CHROM", start.field = "POS", end.field = "end")
  return(grange)

}

apply_qc <- function(vcf, coverage_thresh, SNV = TRUE, AF_thresh = Inf) {

  "
  This function filters variants based on the specified QC metrics. If
  SNV = TRUE, indels are removed. RF_PROB is the Random Forest Probability
  that a called variant is a True Positive. The MacArthur Lab QC process
  uses a RF probability of 0.4 Coverage threshold is the number of total
  alleles that must have been observed (variant or non-variant) for a site.
  In ExAC, the MacArthur Lab uses a cutoff of > 0.8 of the max AN for a site.
  "

  # split X chromosome from others for AN threshold

  thresh = 0.8*coverage_thresh

  nonx <- vcf %>% filter(CHROM != "chrX" & CHROM != "chrY", AN > thresh)
  combined_var <- nonx
  combined_var$AF <- as.numeric(combined_var$AF)

  if (SNV) {
    combined_var <- combined_var %>% filter(nchar(REF) + nchar(ALT) == 2, lcr != 1, decoy != 1)
  } else {
    combined_var <- combined_var %>% filter(lcr != 1, decoy != 1)
  }

  combined_var <- combined_var %>% filter(AF < AF_thresh)
  combined_var$AF <- as.numeric(combined_var$AF)

  combined_var <- combined_var %>% filter(AC > 0)

  return(combined_var)

}

bed_dir <- "/disk2/G4_MS/frequency/MAPS/data"
reference_ranges <- list(g4_gruns = file.path(bed_dir, 'promoter_1000_eG4.gruns.bed'),
                         g4_regions = file.path(bed_dir,'promoter_pG4.info.H3K4me3.tsv'),
             stable_g4_gruns = file.path(bed_dir,'promoter_1000_stable_eG4.gruns.bed'),
             unstable_g4_gruns = file.path(bed_dir,'promoter_1000_unstable_eG4.gruns.bed'),

#                          g4tx_nog4_gruns = file.path(bed_dir,'protein_promoter_1000_noG4.pG4.gruns.g4_tx.bed'),
                         nog4_gruns = file.path(bed_dir,'protein_promoter_1000_noG4.eG4.gruns.bed'),
                         nog4_tx = file.path(bed_dir,'promoter_nopG4.info.H3K4me3.tsv')
                         )

reference_ranges$g4_gruns <- lapply(reference_ranges$g4_gruns, function (x) data.table::fread(
  x,col.names=c("chromosome",'start','end',"txid", "ensembl_gene_id","oristrand")))[[1]]
reference_ranges$g4_regions <- lapply(reference_ranges$g4_regions, function (x) data.table::fread(
  x,col.names=c("chromosome",'start','end',"seq", "oristrand","txid","ensembl_gene_id","oristrand_gene")))[[1]]

## reference_ranges$g4tx_nog4_gruns <- lapply(reference_ranges$g4tx_nog4_gruns, function (x) data.table::fread(
#   x,col.names=c("chromosome",'start','end',"txid", "ensembl_gene_id","oristrand")))[[1]]
reference_ranges$stable_g4_gruns <- lapply(reference_ranges$stable_g4_gruns, function (x) data.table::fread(
  x,col.names=c("chromosome",'start','end',"txid", "ensembl_gene_id","oristrand")))[[1]]
reference_ranges$unstable_g4_gruns <- lapply(reference_ranges$unstable_g4_gruns, function (x) data.table::fread(
  x,col.names=c("chromosome",'start','end',"txid","ensembl_gene_id","oristrand")))[[1]]

reference_ranges$nog4_gruns <- lapply(reference_ranges$nog4_gruns, function (x) data.table::fread(
  x,col.names=c("chromosome",'start','end',"txid", "ensembl_gene_id","oristrand")))[[1]]
reference_ranges$nog4_tx <- lapply(reference_ranges$nog4_tx, function (x) data.table::fread(
  x,col.names=c("chromosome",'start','end',"prmtr_id","ensembl_gene_id", "oristrand","txid")))[[1]]

reference_ranges <- lapply(reference_ranges, function(x) x %>% filter(!chromosome == 'chrY'))
reference_ranges <- lapply(reference_ranges, function(x) mutate(x,txid=substr(txid,1,15)))
reference_ranges <- lapply(reference_ranges, function(x) mutate(x,ensembl_gene_id=substr(ensembl_gene_id,1,15)))
reference_ranges <- lapply(reference_ranges, function(x) mutate(x,end=end-1))
reference_granges <- lapply(reference_ranges, function(x) makeGRangesFromDataFrame(x,
                                                                                   keep.extra.columns = TRUE,
                                                                                   ignore.strand = TRUE,
                                                                                   seqnames.field = "chromosome",
                                                                                   start.field = "start",
                                                                                   end.field = "end",
                                                                                   starts.in.df.are.0based = FALSE))
cds_ranges <- data.table::fread(file.path(bed_dir,'cds_region.used.bed'),
                         col.names = c("chromosome",'start','end',"ensembl_gene_id","gene_name", "oristrand","txid"))
cds_ranges <- cds_ranges %>% filter(!chromosome == 'chrY')
cds_ranges <- mutate(cds_ranges,ensembl_gene_id=substr(ensembl_gene_id,1,15))
cds_ranges <- mutate(cds_ranges,txid=substr(txid,1,15))
cds_ranges$end <- cds_ranges$end -1
cds_ranges <- makeGRangesFromDataFrame(cds_ranges,
                                       keep.extra.columns = TRUE,
                                       ignore.strand = TRUE,
                                       seqnames.field = 'chromosome',
                                       start.field = 'start',
                                       end.field = 'end',
                                       starts.in.df.are.0based = FALSE)
promoter_region <- data.table::fread(file.path(bed_dir,'promoter_info.annotion.H3K4me3.tsv'),
                                     col.names = c("chromosome",'start','end',"prmtr_id","ensembl_gene_id", "oristrand","txid")
)
promoter_region <- mutate(promoter_region,ensembl_gene_id=substr(ensembl_gene_id,1,15))
promoter_region <- mutate(promoter_region,txid=substr(txid,1,15))
promoter_region <- makeGRangesFromDataFrame(promoter_region,
                                       keep.extra.columns = TRUE,
                                       ignore.strand = TRUE,
                                       seqnames.field = 'chromosome',
                                       start.field = 'start',
                                       end.field = 'end',
                                       starts.in.df.are.0based = FALSE)

In [ ]:
# promoter.MAPS.R
library(dplyr)
library(GenomicRanges)
library(ggplot2)
#library(ensembldb)

setwd("/disk2/G4_MS/frequency/MAPS")
source("/disk2/G4_MS/frequency/MAPS/data/tss_stable_unstable.data.get.R")
load("data/RData/ps_lm_genomes.RData")

gtffile <- "/disk2/G4_MS/frequency/MAPS/data/Homo_sapiens.GRCh38.104.gtf.gz"
DB.104 <- ensembldb::ensDbFromGtf(gtf = gtffile)
EnsDb.Hsapiens.v104 <- ensembldb::EnsDb(DB.104)
## cds rangeget
mu <- data.table::fread("data/mutation_rate_methylation_bins.txt")
mu <- mu %>%
  mutate(ref_context = paste0(context,":",methylation_level),
         mut_context = paste0(substr(context,1,1),
                              alt,
                              substr(context,3,3)),
         mutation = paste0(ref_context,">",mut_context))
## Genes with alternativate pG4 isoform
dlata_g4 <- unique(intersect(reference_ranges$nog4_tx$ensembl_gene_id,reference_ranges$g4_regions$ensembl_gene_id))
constitutive_g4 <-unique(setdiff(reference_ranges$g4_regions$ensembl_gene_id,reference_ranges$nog4_tx$ensembl_gene_id))

# load constraint table by gene
gene_constraints <- data.table::fread("/disk2/G4_MS/reference/gnomad.v2.1.1.lof_metrics.by_gene.txt")
non_plof_genes <- gene_constraints[order(gene_constraints$oe_lof_upper,decreasing = TRUE),]
non_plof_genes <- non_plof_genes[1:6568,]
plof_genes <- gene_constraints[order(gene_constraints$oe_lof_upper,decreasing = FALSE),]
plof_genes <- plof_genes[1:6568,]
clinvar_genes <- data.table::fread("/disk2/G4_MS/reference/clinvar_20220115.vcf_annotated_af.tsv")
clinvar_genes <- AnnotationDbi::mapIds(EnsDb.Hsapiens.v104,
                                       keys=gsub(pattern = ":.*","",clinvar_genes$GENEINFO),
                                       column = "GENEID",keytype="SYMBOL")
cgcs <- data.table::fread("/disk2/G4_MS/reference//cancer_gene_census.tsv")
cgcs <- AnnotationDbi::mapIds(EnsDb.Hsapiens.v104,keys=cgcs$GeneSymbol,column = "GENEID",keytype="SYMBOL")

tsgs <- data.table::fread("/disk2/G4_MS/reference//tsgenes.tsv")
# tsgs <- tsgs[which(tsgs$GeneType=='protein-coding'),]
tsgs <- AnnotationDbi::mapIds(EnsDb.Hsapiens.v104,keys=tsgs$GeneSymbol,column = "GENEID",keytype="SYMBOL")

# Load genome-wide methylation estimates
load('data/methylation_hg38.RData')
promoter_gnomad <- data.table::fread("data/promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv")
colnames(promoter_gnomad) <- c("CHROM", "POS", "REF", "ALT", "AN", "AC", "AF")
promoter_hept <- data.table::fread("data/promoter_region_gnomad.genomes.v3.1.sites.pass_snps.seq", header = FALSE)
promoter_tri <- substr(promoter_hept$V2,3,5)
promoter_gnomad$context <- toupper(promoter_tri)
promoter_gnomad$mut_context <- paste0(substr(promoter_gnomad$context,1,1),
                                      promoter_gnomad$ALT,
                                      substr(promoter_gnomad$context,3,3))
promoter_gnomad$hept <- toupper(promoter_hept$V2)
promoter_gnomad <- promoter_gnomad %>%  select(CHROM, POS, REF, ALT, AN, AC, AF, context, mut_context, hept) %>%
  filter(CHROM != 'chrX', AN > 0.8*152312, AC >= 1)
rm(promoter_hept)
rm(promoter_tri)
promoter_gnomad_grange <- granges_from_vcf(promoter_gnomad)
### 
# combined_promoter_variants = promoter_gnomad_grange
g1_allowed <- c('AGG',
                'CCA',
                'CCG',
                'CCT',
                'CGG',
                'TGG')
g2_allowed <- c('CCC','GGG')
g3_allowed <- c('ACC',
                'GCC',
                'GGA',
                'GGC',
                'GGT',
                'TCC')

allowed <- list('pos1' = g1_allowed,
                'pos2' = g2_allowed,
                'pos3' = g3_allowed)

######################
## Helper Functions ##
######################

get_ps <- function(vcf_list, mu) {

  n <- lapply(vcf_list, function(x) table(x$mutation))
  ps <- lapply(vcf_list, function(x) table(x[x$AC == 1]$mutation))

  ps_res <- mapply(function(x, y) cbind(x, y[match(names(x), names(y))]), x=n, y=ps, SIMPLIFY = FALSE)
  ps_res <- lapply(ps_res, function(x) x[match(mu$mutation, rownames(x)),])

  return(ps_res)
}


get_mu_avg <- function(ps_list, mu) {

  res <- lapply(ps_list, function(x) sum((x*mu$mu_snp), na.rm = TRUE) / sum(x, na.rm = TRUE))


  return(res)

}


calc_maps <- function(raw_ps, ps_counts, predictions) {

  obs <- unlist(raw_ps) * unlist(ps_counts)
  exp <- unlist(predictions) * unlist(ps_counts)

  maps <- (obs - exp) / unlist(ps_counts)

  return(maps)

}
# MAPS 2 model:
calc_maps2 <- function(variants, mu, ps_lm) {

  variants <- data.frame(variants)

  total_variants <- dim(variants)[[1]]

  variants <- split(variants, variants$group)

  if (length(variants) == 2) {
    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)
    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)

  } else if (names(variants) == 'no_adjust') {

    methyl_adjust <- c('exp_singletons.1' = 0,
                       'obs_singletons' = 0)
    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)
  } else {
    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)
    no_adjust <- c('exp_singletons.1' = 0,
                   'obs_singletons' = 0)
  }
  combined <- methyl_adjust + no_adjust

  maps <- (combined['obs_singletons'] - combined['exp_singletons.1']) / total_variants
  raw_p <- combined['obs_singletons'] / total_variants
  se <-  sqrt(raw_p*(1-raw_p)/total_variants)
  result <- c("MAPS" = as.numeric(maps), "SE" = as.numeric(se), "N" = as.integer(total_variants))
  return(result)

}

get_obs_exp_singletons <- function(variants, ps_lm) {

  variants <- na.omit(variants)
  mu_avg <- sum(variants$n * variants$mu_snp) / sum(variants$n)
  exp_singletons <- sum(variants$n) * predict(ps_lm, data.frame('mu_snp' = mu_avg))
  obs_singletons <- sum(variants$singletons)
  return(c('exp_singletons' = exp_singletons,
           'obs_singletons' = obs_singletons))

}

do_MAPS <- function(vcf_list, mu, ps_model) {
  # determine proportion singletons
  ps <- get_ps(vcf_list, mu)
  # determine mu average for the variants + contexts
  mu_avg <- get_mu_avg(ps, mu)
  # extract raw proportion of singletons
  raw_p <- lapply(vcf_list, function(x) length(x[x$AC==1])/length(x))
  var_counts <- lapply(vcf_list, function(x) length(x))
  se <- mapply(function(p, n ) sqrt(p*(1-p)/n), raw_p, var_counts)
  # predicted ps:
  ps_predict <- lapply(mu_avg, function(x) predict(ps_model, data.frame(mu = unlist(x))))
  maps <- calc_maps(raw_p, var_counts, ps_predict)
  return(maps)

}

annotate_grun_position <- function(grun_grange, promoter_region, width = 3) {

  gruns <- grun_grange[width(grun_grange) == width]
  gruns_df <- data.frame(gruns)
  #gruns$strand <- gruns$oristrand
  gruns_df$strand <- promoter_region$oristrand[match(gruns_df$txid, promoter_region$txid)]
  # gruns_df$strand <- gruns_df$strand.1
  # gruns_df <- gruns_df[,!colnames(gruns_df) %in% c("oristrand","strand.1")]
  tile <- makeGRangesFromDataFrame(gruns_df)
  tile <- tile(tile, width = 1) %>% as.data.frame()
  pos_tiles <- tile %>% filter(strand == '+')
  neg_tiles <- tile %>% filter(strand == '-')

  pos_tiles$var_pos <- rep(x = c(1:width), dim(pos_tiles)[[1]]/width)
  neg_tiles$var_pos <- rep(x = c(1:width), dim(neg_tiles)[[1]]/width)

  tiles <- rbind(pos_tiles, neg_tiles)

  pos1 <- tiles %>% filter(var_pos == 1)
  pos2 <- tiles %>% filter(var_pos == 2)
  pos3 <- tiles %>% filter(var_pos == 3)

  res <- list("pos1" = pos1,
              "pos2" = pos2,
              "pos3" = pos3)
  res <- lapply(res, function(x) makeGRangesFromDataFrame(x, keep.extra.columns = TRUE))
  return(res)

}
match_var_to_strand <- function(var_grange_first, var_grange_last, tx_grange) {

  # this function annotates a variant as being either first or last in a trinucleotide G/C run
  # based on the strand of the UTR transcript

  pos_strand <- tx_grange[tx_grange$oristrand == '+']
  neg_strand <- tx_grange[tx_grange$oristrand == '-']
  pos_strand_first <- subsetByOverlaps(var_grange_first, pos_strand)
  pos_strand_last <- subsetByOverlaps(var_grange_last, pos_strand)

  neg_strand_first <- subsetByOverlaps(var_grange_last, neg_strand)
  neg_strand_last <- subsetByOverlaps(var_grange_first, neg_strand)

  first_vars <- c(pos_strand_first, neg_strand_first)
  last_vars <- c(pos_strand_last, neg_strand_last)

  return(list('pos1'=first_vars,
              'pos3'=last_vars))

}

calc_maps2_bootstrap <- function(variants, mu, ps_lm) {

  total_variants <- dim(variants)[[1]]

  variants <- variants[sample(1:total_variants, total_variants, replace = TRUE),]

  variants <- split(variants, variants$group)

  if (length(variants) == 2) {

    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)

    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)

  } else if (names(variants) == 'no_adjust') {

    methyl_adjust <- c('exp_singletons.1' = 0,
                       'obs_singletons' = 0)

    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)

  } else {

    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)

    no_adjust <- c('exp_singletons.1' = 0,
                   'obs_singletons' = 0)
  }
combined <- methyl_adjust + no_adjust

  maps <- (combined['obs_singletons'] - combined['exp_singletons.1']) / total_variants

  return(as.numeric(maps))

}
                
bootstrap_maps2 <- function(variants, mu, ps_lm, ntimes = 10000) {

  variants <- data.frame(variants)

  variants <- variants %>%
    select(AC, AN, group, mut)

  res <- vector("list", length = ntimes)

  res <- lapply(res, function(x) calc_maps2_bootstrap(variants, mu, ps_lm))

  return(unlist(res))

}

resample_MAPS <- function(vcf, mu, ps_model) {

  n <- table(vcf$mut_context)
  ps <- table(vcf[vcf$AC == 1]$mut_context)

  ps <- cbind(n, ps[match(names(n), names(ps))])
  ps <- ps[match(mu$mut_context, rownames(ps)),]

  mu_average <- sum(ps*mu$mu_snp, na.rm =TRUE) / sum(ps, na.rm = TRUE)

  raw_p <- length(vcf[vcf$AC==1])/length(vcf)
  var_counts <- length(vcf)
  ps_predict <- predict(ps_model, data.frame(mu = unlist(mu_average)))

  obs <- unlist(raw_p) * unlist(var_counts)
  exp <- unlist(ps_predict) * unlist(var_counts)
  maps <- (obs-exp)/unlist(var_counts)

  return(maps)

}

generate_bootstrap <- function(vcf, n, mu, ps_model) {

  x <- length(vcf)

  res <- rep(x, n)

  samples <- lapply(res, function(x) sample(x, size = x, replace = TRUE))

  maps_bootstrap <- lapply(samples, function(x) resample_MAPS(vcf[x], mu = mu, ps_model = ps_model))

  return(unlist(maps_bootstrap))

}
                
chunked_bootstrap <- function(vcf, n, mu, ps_model) {

  chunksize <- as.integer(n/10)
  chunk1 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk2 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk3 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk4 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk5 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk6 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk7 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk8 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk9 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk10 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                             mu = mu, ps_lm = ps_model)
  result <- c(chunk1, chunk2, chunk3, chunk4, chunk5,
              chunk6, chunk7, chunk8, chunk9, chunk10)
  return(result)

}
annotate_methylation <- function(variants, mutation_table, methylation) {

  variants <- left_join(variants, methylation)
  # set missing methylation values to zero
  variants$methyl_bins[is.na(variants$methyl_bins)] <- 0
  fixed_context <- ifelse(variants$context %in% mutation_table$context,
                          variants$context,
                          chartr('ATGC','TACG',variants$context))
  fixed_mut <- ifelse(variants$context %in% mutation_table$context,
                      variants$mut_context,
                      chartr('ATGC','TACG',variants$mut_context))

  # amend mutation context for MAPS score calculation
  variants$mut <- paste0(fixed_context,':',
                         variants$methyl_bins,
                         '>',
                         fixed_mut)
  variants$mut <- ifelse(variants$mut %in% mutation_table$mutation,
                         variants$mut,
                         gsub('[1|2]',
                              '0',
                              variants$mut))
  variants$context <- fixed_context
  variants$mut_context <- fixed_mut

  # annotate variants for methylation adjustment
  var_adjust <- mutation_table %>%
    filter(methylation_level != 0) %>%
    select(context) %>% unlist()
  variants$group <- ifelse((variants$context %in% var_adjust),
                           ifelse((variants$REF == 'C' & variants$ALT == 'T') | (variants$REF == 'G' & variants$ALT == 'A'),
                                  'methyl_adjust',
                                  'no_adjust'),
                           'no_adjust')

  return(variants)

}

##############
## Analysis ##
##############
# annotate methylation context for MAPS analysis
# correct gruns
reference_granges$g4_gruns <- subsetByOverlaps(reference_granges$g4_gruns, promoter_region)
## combined_promoter_variants = promoter_gnomad_grange
promoter_gnomad_grange_df <- data.frame(promoter_gnomad_grange) %>% mutate(CHROM = seqnames, POS = start)
promoter_gnomad_grange_df <- annotate_methylation(data.frame(promoter_gnomad_grange_df),
                                                  mu,
                                                  combined_methylation_estimates)
promoter_gnomad_grange <- promoter_gnomad_grange_df %>% select(-c(CHROM, POS)) %>%
  makeGRangesFromDataFrame(., keep.extra.columns = TRUE)

# subset pG4s by alternative vs. constitutive
promoter_delta_pg4 <- reference_granges$g4_gruns[reference_granges$g4_gruns$ensembl_gene_id %in% dlata_g4]
promoter_const_pg4 <- reference_granges$g4_gruns[!reference_granges$g4_gruns$ensembl_gene_id %in% dlata_g4]
                           
promoter_stable_pg4 <- reference_granges$stable_g4_gruns
#promoter_stable_pg4 <- promoter_stable_pg4[!promoter_stable_pg4$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
promoter_unstable_pg4 <-reference_granges$unstable_g4_gruns
# promoter_unstable_pg4 <- promoter_unstable_pg4[!promoter_unstable_pg4$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
# Annotate grun positions (G1, G2, G3) and remove sequence contexts that are mismatched
promoter_g4_gruns_positional <- annotate_grun_position(reference_granges$g4_gruns,promoter_region)
promoter_g4_vars <- lapply(promoter_g4_gruns_positional, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# repeat for delta pG4
promoter_delta_pg4 <- annotate_grun_position(promoter_delta_pg4,promoter_region)
promoter_delta_pg4 <- lapply(promoter_delta_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# repeat for constitutive pG4
promoter_const_pg4 <- annotate_grun_position(promoter_const_pg4,promoter_region)
promoter_const_pg4 <- lapply(promoter_const_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))                     

# repeat for stable pG4
promoter_stable_pg4 <- annotate_grun_position(promoter_stable_pg4,promoter_region)
promoter_stable_pg4 <- lapply(promoter_stable_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# repeat for unstable pG4
promoter_unstable_pg4 <- annotate_grun_position(promoter_unstable_pg4,promoter_region)
promoter_unstable_pg4 <- lapply(promoter_unstable_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# combine variant sets and perform MAPS
promoter_combined <- c('G1' = promoter_g4_vars$pos1,
                  'G2' = promoter_g4_vars$pos2,
                  'G3' = promoter_g4_vars$pos3
)
promoter_combined_delta <- c('G1' = promoter_delta_pg4$pos1,
                             'G2' = promoter_delta_pg4$pos2,
                             'G3' = promoter_delta_pg4$pos3
)
promoter_combined_const <- c('G1' = promoter_const_pg4$pos1,
                             'G2' = promoter_const_pg4$pos2,
                             'G3' = promoter_const_pg4$pos3
)
###
promoter_combined_stable <- c('G1' = promoter_stable_pg4$pos1,
                             'G2' = promoter_stable_pg4$pos2,
                             'G3' = promoter_stable_pg4$pos3
)
promoter_combined_unstable <- c('G1' = promoter_unstable_pg4$pos1,
                             'G2' = promoter_unstable_pg4$pos2,
                             'G3' = promoter_unstable_pg4$pos3
)
# Evaluate MAPS
maps_pg4_combined <- lapply(promoter_combined, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_delta <- lapply(promoter_combined_delta, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_const<- lapply(promoter_combined_const, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_stable <- lapply(promoter_combined_stable, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_unstable <- lapply(promoter_combined_unstable, function(x) calc_maps2(x, mu, ps_lm_genomes))                                       

######################################################################
## repeat for GGG/CCC repeats to control for local sequence context ##
######################################################################

promoter_combined_npg4 <- subsetByOverlaps(promoter_gnomad_grange, promoter_combined$G1, invert = TRUE)
promoter_combined_npg4 <- subsetByOverlaps(promoter_combined_npg4, promoter_combined$G2, invert = TRUE)
promoter_combined_npg4 <- subsetByOverlaps(promoter_combined_npg4, promoter_combined$G3, invert = TRUE)
# remove CDS
promoter_combined_npg4 <- subsetByOverlaps(promoter_combined_npg4, cds_ranges, invert = TRUE)
# remove CpG sites
promoter_combined_npg4_central <- grepl('[ATG]CCC[ATG]|[ATC]GGG[ATC]', substr(promoter_combined_npg4$hept, 2, 6))
promoter_combined_npg4_central <- promoter_combined_npg4[promoter_combined_npg4_central]

# Map 1st and 3rd positions for non-pG4 sequences:
promoter_region_df <- data.frame(promoter_region)
promoter_region_df$strand <- promoter_region_df$oristrand
# promoter_region_df <- promoter_region_df[,!colnames(promoter_region_df) %in% c("oristrand")]
promoter_region_df <- makeGRangesFromDataFrame(promoter_region_df,
                                               keep.extra.columns = TRUE,
                                               seqnames.field = 'seqnames',
                                               start.field = 'start',
                                               end.field = 'end')

promoter_combined_npg4_first <- grepl('[ATG]CCC[ATG]|[ATC]GGG[ATC]', substr(promoter_combined_npg4$hept, 3, 7))
promoter_combined_npg4_first <- promoter_combined_npg4[promoter_combined_npg4_first]

promoter_combined_npg4_last <- grepl('[ATG]CCC[ATG]|[ATC]GGG[ATC]', substr(promoter_combined_npg4$hept, 1, 5))
promoter_combined_npg4_last <- promoter_combined_npg4[promoter_combined_npg4_last]

first_last_npg4 <- match_var_to_strand(promoter_combined_npg4_first, promoter_combined_npg4_last,
                                       promoter_region_df)

promoter_nog4_combined <- list('pos1' = first_last_npg4$pos1,
                      'pos2' = promoter_combined_npg4_central,
                      'pos3' = first_last_npg4$pos3)
# All non-pG4 GGG / CCC contexts
maps_nong4_first_last <- lapply(promoter_nog4_combined, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_all_promoter <- list('all'=calc_maps2(promoter_gnomad_grange, mu, ps_lm_genomes))

# non-pG4 delta GGG /CCC contexts
promoter_nog4_combined_delta <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% dlata_g4]
                                                                          ))
maps_promoter_nog4_delta <- lapply(promoter_nog4_combined_delta, function(x) calc_maps2(x, mu, ps_lm_genomes))

all_promoter_delta_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% dlata_g4]
)
all_promoter_delta_variants <- subsetByOverlaps(all_promoter_delta_variants,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$g4_regions$ensembl_gene_id]
                                                )
maps_all_promoter_delta <- list('all'=calc_maps2(all_promoter_delta_variants, mu, ps_lm_genomes))
                                
# non-pG4 const GGG /CCC contexts
promoter_nog4_combined_const <- lapply(promoter_nog4_combined, function(x)
  subsetByOverlaps(x, promoter_region_df[!promoter_region_df$ensembl_gene_id %in% dlata_g4]
))
promoter_nog4_combined_const <- lapply(promoter_nog4_combined_const, function(x)
  subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$g4_regions$ensembl_gene_id]
  )
)

maps_promoter_nog4_const <- lapply(promoter_nog4_combined_const, function(x) calc_maps2(x, mu, ps_lm_genomes))
# select variants that are not in delta pG4 genes' promoter
const_all_promoter_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           promoter_region_df[!promoter_region_df$ensembl_gene_id %in% dlata_g4])
# subset these variants by those that are in pG4 genes to isolate "constitutive" pG4s
const_all_promoter_variants <- subsetByOverlaps(const_all_promoter_variants,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$g4_regions$ensembl_gene_id]
)
maps_all_promoter_const <- list('all'=calc_maps2(const_all_promoter_variants, mu, ps_lm_genomes))

# non-pG4 stable GGG/CCC contexts
promoter_nog4_combined_stable <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
                                                                          ))
promoter_nog4_combined_stable <- lapply(promoter_nog4_combined_stable, function(x) subsetByOverlaps(x, promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
                                                                          ))
maps_promoter_nog4_stable <- lapply(promoter_nog4_combined_stable, function(x) calc_maps2(x, mu, ps_lm_genomes))

all_promoter_stable_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           reference_granges$stable_g4_gruns
)
#all_promoter_stable_variants <- subsetByOverlaps(all_promoter_stable_variants,
#                                           promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
#)
maps_all_promoter_stable <- list('all'=calc_maps2(all_promoter_stable_variants, mu, ps_lm_genomes))

# non-pG4 unstable GGG/CCC contexts
promoter_nog4_combined_unstable <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
                                                                          ))
promoter_nog4_combined_unstable <- lapply(promoter_nog4_combined_unstable, function(x) subsetByOverlaps(x, promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
                                                                          ))
maps_promoter_nog4_unstable <- lapply(promoter_nog4_combined_unstable, function(x) calc_maps2(x, mu, ps_lm_genomes))

all_promoter_unstable_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           reference_granges$unstable_g4_gruns)
#all_promoter_unstable_variants <- subsetByOverlaps(all_promoter_unstable_variants,
#                                           promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
#)
maps_all_promoter_unstable <- list('all'=calc_maps2(all_promoter_unstable_variants, mu, ps_lm_genomes)) 
                                      
####################################################################
## Evaluate MAPS for variants in top 1/3 of most constrained genes ##
#####################################################################
## promoter_g4_gruns = combined_g4_gruns
promoter_g4_gruns <- reference_granges$g4_gruns[width(reference_granges$g4_gruns) == 3]
promoter_g4_gruns$plof <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% plof_genes$gene_id,
                                 TRUE,
                                 FALSE)

promoter_combined_g4_plof_gruns <- lapply(promoter_combined,
                                 function(x) subsetByOverlaps(
                                   x, promoter_g4_gruns[promoter_g4_gruns$plof == TRUE])
                                 )

# overlap gnomAD variants
combined_g4_plof_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$plof == TRUE]))

#combined_g4_nonplof_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$non_plof == TRUE]))

# evaluate MAPS for top third most constrained UTR
maps_plof_gruns <-  lapply(combined_g4_plof_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))

# Evaluate all variants in top 1/3 most constrained UTR
promoter_reference_grange <- data.frame(promoter_region) %>%
  select(txid,
         ensembl_gene_id,
         start,
         end,
         oristrand,
         seqnames)
promoter_reference_grange$strand <- promoter_reference_grange$oristrand
promoter_reference_grange$top_third <- ifelse(promoter_reference_grange$ensembl_gene_id %in% plof_genes$gene_id, TRUE, FALSE)
promoter_reference_grange$bottom_third <- ifelse(promoter_reference_grange$ensembl_gene_id %in% non_plof_genes$gene_id, TRUE,FALSE)
promoter_reference_grange$clinvar_signal <- ifelse(promoter_reference_grange$ensembl_gene_id %in% clinvar_genes, TRUE, FALSE)
promoter_reference_grange$cgcs_signal <- ifelse(promoter_reference_grange$ensembl_gene_id %in% cgcs, TRUE, FALSE)
promoter_reference_grange$tsgs_signal <- ifelse(promoter_reference_grange$ensembl_gene_id %in% tsgs, TRUE, FALSE)
promoter_reference_grange <- promoter_reference_grange %>% makeGRangesFromDataFrame(start.field = 'start',
                                                                            end.field = 'end',
                                                                            seqnames.field = 'seqnames', keep.extra.columns =TRUE)

# define variant subsets
all_promoter_plof <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$top_third == TRUE])
all_promoter_nonplof <- subsetByOverlaps(promoter_gnomad_grange,promoter_reference_grange[promoter_reference_grange$bottom_third == TRUE])
all_promoter_clinvar_signal <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$clinvar_signal == TRUE])
all_promoter_cgcs_signal <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$cgcs_signal == TRUE])
all_promoter_tsgs_signal <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$tsgs_signal == TRUE])

nog4_combined_plof <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$top_third == TRUE]))
nog4_combined_clinvar_signal <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$clinvar_signal == TRUE]))
nog4_combined_cgcs_signal <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$cgcs_signal == TRUE]))
nog4_combined_tsgs_signal <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$tsgs_signal == TRUE]))


promoter_g4_gruns$clinvar_signal <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% clinvar_genes, TRUE, FALSE)
promoter_g4_gruns$cgcs_signal <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% cgcs, TRUE, FALSE)
promoter_g4_gruns$tsgs_signal <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% tsgs, TRUE, FALSE)

#####################################
## evaluate MAPS for ClinVar genes ##
#####################################
maps_plof_all_variants <- list('all_promoter_plof' = calc_maps2(all_promoter_plof, mu, ps_lm_genomes))

maps_nog4_combined_plof <- lapply(nog4_combined_plof, function(x) calc_maps2(x, mu, ps_lm_genomes))

maps_clinvar_signal_all_variants <- list('clinvar' = calc_maps2(all_promoter_clinvar_signal, mu, ps_lm_genomes))
maps_cgcs_signal_all_variants <- list('cgcs' = calc_maps2(all_promoter_cgcs_signal, mu, ps_lm_genomes))
maps_tsgs_signal_all_variants <- list('tsgs' = calc_maps2(all_promoter_tsgs_signal, mu, ps_lm_genomes))

combined_g4_clinvar_signal_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$clinvar_signal == TRUE]))
combined_g4_cgcs_signal_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$cgcs_signal == TRUE]))
combined_g4_tsgs_signal_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$tsgs_signal == TRUE]))

maps_clinvar_pg4 <- lapply(combined_g4_clinvar_signal_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_cgcs_pg4 <- lapply(combined_g4_cgcs_signal_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_tsgs_pg4 <- lapply(combined_g4_tsgs_signal_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))

maps_clinvar_nong4 <- lapply(nog4_combined_clinvar_signal, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_cgcs_nong4 <- lapply(nog4_combined_cgcs_signal, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_tsgs_nong4 <- lapply(nog4_combined_tsgs_signal, function(x) calc_maps2(x, mu, ps_lm_genomes))


combined_g4_clinvar_signal_delta_gruns <- mapply(function(x,y) subsetByOverlaps(x,y),
                                         x = combined_g4_clinvar_signal_gruns,
                                         y = promoter_combined_delta)
combined_g4_cgcs_signal_delta_gruns <- mapply(function(x,y) subsetByOverlaps(x,y),
                                         x = combined_g4_cgcs_signal_gruns,
                                         y = promoter_combined_delta)
combined_g4_tsgs_signal_delta_gruns <- mapply(function(x,y) subsetByOverlaps(x,y),
                                         x = combined_g4_tsgs_signal_gruns,
                                         y = promoter_combined_delta)


maps_clinvar_pg4_delta <- lapply(combined_g4_clinvar_signal_delta_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_cgcs_pg4_delta <- lapply(combined_g4_cgcs_signal_delta_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_tsgs_pg4_delta <- lapply(combined_g4_tsgs_signal_delta_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))

# Generate MAPS estimates for missense and putative loss of function-causing variants in gnomAD
missense <- data.table::fread('data/gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv',header=FALSE)
colnames(missense) <- c('CHROM','POS','REF','ALT','AN','AC','AF','type','other', 'vep')
heptamers <- data.table::fread("data/gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.seq", header = FALSE)
missense$heptamers <- toupper(heptamers$V2)
missense$context <- toupper(substr(heptamers$V2, 3, 5))

rm(heptamers)
                              
an_adj <- 0.8*152312
missense <- missense %>% filter(AC != 0,
                                AN >= an_adj, CHROM != "chrX") %>%
                        select(-c(type, other, vep))

missense$mut_context <- paste0(substr(missense$context, 1, 1),
                               missense$ALT,
                               substr(missense$context, 3, 3))

missense$CHROM <- as.character(missense$CHROM)
missense <- annotate_methylation(missense, mu, combined_methylation_estimates)
plof <- data.table::fread("data/gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv",
                          header=FALSE,
                          col.names = c('CHROM','POS','REF','ALT','AN','AC','AF','type','other', 'vep')
)
heptamers <- data.table::fread('data/gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv.seq', header = FALSE)
plof$heptamers <- heptamers$V2 %>% toupper()
rm(heptamers)
plof <- plof %>% filter(AC >= 1,
                        AN >= 0.8*152312, CHROM != "chrX",
                        nchar(REF) == 1, nchar(ALT) == 1) %>%
  select(-c(type, other, vep))

plof$context <- substr(plof$heptamers, 3,5)

plof$mut_context <- paste0(substr(plof$context, 1, 1),
                           plof$ALT,
                           substr(plof$context, 3, 3))

plof$CHROM <- as.character(plof$CHROM)
plof <- annotate_methylation(plof, mu, combined_methylation_estimates)
####
cds_varsets <- list('missense_all' = missense,
                    'lof_all' = plof)
maps_cds_markers <- lapply(cds_varsets, function(x) calc_maps2(x, mu, ps_lm_genomes))
##############################
## Generate bootstrapped CI ##
##############################
# G4_combined bootstraps
g4_combined_bootstraps <- lapply(promoter_combined, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))
save(g4_combined_bootstraps, file = 'RData/g4_positional_maps_bootstraps_ncpg.RData')
lapply(g4_combined_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/g4_positional_maps_bootstraps_ncpg.RData')

G4 delta bootstraps
g4_combined_delta_bootstraps <- lapply(promoter_combined_delta, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000)) #lapply(utr_combined_delta, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(g4_combined_delta_bootstraps, file = 'RData/g4_combined_delta_bootstraps.RData')
lapply(g4_combined_delta_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load("RData/g4_combined_delta_bootstraps.RData")

# G4 const bootstraps
g4_combined_const_bootstraps <- lapply(promoter_combined_const, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(utr_combined_const, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(g4_combined_const_bootstraps, file = 'RData/g4_combined_const_bootstraps.RData')
lapply(g4_combined_const_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load("RData/g4_combined_const_bootstraps.RData")
       
# G4 stable bootstraps
g4_combined_stable_bootstraps <- lapply(promoter_combined_stable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000)) 
save(g4_combined_stable_bootstraps, file = 'RData/g4_combined_stable_bootstraps.RData')
#lapply(g4_combined_stable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
#load("RData/g4_combined_stable_bootstraps.RData")

# G4 unstable bootstraps
g4_combined_unstable_bootstraps <- lapply(promoter_combined_unstable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))
save(g4_combined_unstable_bootstraps, file = 'RData/g4_combined_unstable_bootstraps.RData')
#lapply(g4_combined_unstable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
#load("RData/g4_combined_unstable_bootstraps.RData")

# # non-G4 G bootstraps
nog4_combined_bootstraps <- lapply(promoter_nog4_combined, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_bootstraps, file = 'RData/nog4_combined_bootstraps.RData')
#lapply(nog4_combined_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
load("RData/nog4_combined_bootstraps.RData")

# # non-G4 delta-G bootstraps
nog4_combined_delta_bootstraps <-  lapply(promoter_nog4_combined_delta, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_delta, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_delta_bootstraps, file = 'RData/nog4_combined_delta_bootstraps.RData')
lapply(nog4_combined_delta_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_delta_bootstraps.RData')

# # non-G4 const-G bootstraps
nog4_combined_const_bootstraps <-  lapply(promoter_nog4_combined_const, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_const, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_const_bootstraps, file = 'RData/nog4_combined_const_bootstraps.RData')
#lapply(nog4_combined_const_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_const_bootstraps.RData')

# non-G4 stable-G bootstraps
nog4_combined_stable_bootstraps <-  lapply(promoter_nog4_combined_stable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_delta, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_stable_bootstraps, file = 'RData/nog4_combined_stable_bootstraps.RData')
#lapply(nog4_combined_stable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
#load('RData/nog4_combined_stable_bootstraps.RData')

# non-G4 unstable-G bootstraps
nog4_combined_unstable_bootstraps <-  lapply(promoter_nog4_combined_unstable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_delta, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_unstable_bootstraps, file = 'RData/nog4_combined_unstable_bootstraps.RData')
#lapply(nog4_combined_unstable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
#load('RData/nog4_combined_unstable_bootstraps.RData')       
       
# # g4 in PLOF genes bootstrap
combined_g4_plof_gruns_bootstrap <-  lapply(promoter_combined_g4_plof_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_plof_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_plof_gruns_bootstrap, file = 'RData/combined_g4_plof_gruns_bootstraps.RData')
#lapply(combined_g4_plof_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
load("RData/combined_g4_plof_gruns_bootstraps.RData")

# # nonG4 in PLOF genes bootstrap
combined_npg4_plof_gruns_bootstrap <-  lapply(nog4_combined_plof, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_plof, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_npg4_plof_gruns_bootstrap, file = 'RData/combined_npg4_plof_gruns_bootstrap.RData')
lapply(combined_npg4_plof_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_npg4_plof_gruns_bootstrap.RData')
# #
# G4 in disease gene bootstrap
combined_g4_clinvar_signal_bootstrap <-  lapply(combined_g4_clinvar_signal_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_clinvar_signal_bootstrap, file = 'RData/combined_g4_clinvar_signal_bootstrap.RData')
lapply(combined_g4_clinvar_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_clinvar_signal_bootstrap.RData')

# # G4 in cgcs gene bootstrap
combined_g4_cgcs_signal_bootstrap <-  lapply(combined_g4_cgcs_signal_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_cgcs_signal_bootstrap, file = 'RData/combined_g4_cgcs_signal_bootstrap.RData')
lapply(combined_g4_cgcs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_cgcs_signal_bootstrap.RData')

# # G4 in disease gene bootstrap
combined_g4_tsgs_signal_bootstrap <-  lapply(combined_g4_tsgs_signal_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_tsgs_signal_bootstrap, file = 'RData/combined_g4_tsgs_signal_bootstrap.RData')
lapply(combined_g4_tsgs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_tsgs_signal_bootstrap.RData')

# nonG4 in disease gene bootstrap
nog4_combined_clinvar_signal_bootstrap <-  lapply(nog4_combined_clinvar_signal, function(x) bootstrap_maps2(x, mu = mu, ps_lm= ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_signal, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_clinvar_signal_bootstrap, file = 'RData/nog4_combined_clivar_signal_bootstrap.RData')
lapply(nog4_combined_clinvar_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_clivar_signal_bootstrap.RData')
# #
# nonG4 in disease gene bootstrap
nog4_combined_cgcs_signal_bootstrap <-  lapply(nog4_combined_cgcs_signal, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_signal, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_cgcs_signal_bootstrap, file = 'RData/nog4_combined_cgcs_signal_bootstrap.RData')
lapply(nog4_combined_cgcs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
load('RData/nog4_combined_cgcs_signal_bootstrap.RData')
# #
# nonG4 in disease gene bootstrap
nog4_combined_tsgs_signal_bootstrap <-  lapply(nog4_combined_tsgs_signal, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_signal, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_tsgs_signal_bootstrap, file = 'RData/nog4_combined_tsgs_signal_bootstrap.RData')
lapply(nog4_combined_tsgs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_tsgs_signal_bootstrap.RData')

# delta g4 in disease genes bootstrap
combined_g4_clinvar_signal_delta_gruns_bootstrap <-  lapply(combined_g4_clinvar_signal_delta_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_delta_gruns,
save(combined_g4_clinvar_signal_delta_gruns_bootstrap, file = 'RData/combined_g4_clinvar_signal_delta_gruns_bootstrap.RData')
lapply(combined_g4_clinvar_signal_delta_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_clinvar_signal_delta_gruns_bootstrap.RData')

# delta g4 in cgcs genes bootstrap
combined_g4_cgcs_signal_delta_gruns_bootstrap <-  lapply(combined_g4_cgcs_signal_delta_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_delta_gruns,
save(combined_g4_cgcs_signal_delta_gruns_bootstrap, file = 'RData/combined_g4_cgcs_signal_delta_gruns_bootstrap.RData')
lapply(combined_g4_cgcs_signal_delta_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_cgcs_signal_delta_gruns_bootstrap.RData')
# 
# # delta g4 in tsgs genes bootstrap
combined_g4_tsgs_signal_delta_gruns_bootstrap <-  lapply(combined_g4_tsgs_signal_delta_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_delta_gruns,
save(combined_g4_tsgs_signal_delta_gruns_bootstrap, file = 'RData/combined_g4_tsgs_signal_delta_gruns_bootstrap.RData')
lapply(combined_g4_tsgs_signal_delta_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_tsgs_signal_delta_gruns_bootstrap.RData')

# ALL promoter variants bootstrap
promoter_gnomad_grange_bootstrap <- chunked_bootstrap(promoter_gnomad_grange, 10000, mu = mu, ps_model = ps_lm_genomes)
save(promoter_gnomad_grange_bootstrap, file = 'RData/promoter_gnomad_grange_bootstrap.RData')
quantile(promoter_gnomad_grange_bootstrap, c(0.05, 0.95))
load('RData/promoter_gnomad_grange_bootstrap.RData')

# # ALL promoter variants delta pG4 bootstrap
combined_promoter_delta_variants_bootstrap <- chunked_bootstrap(all_promoter_delta_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_delta_variants_bootstrap, file = 'RData/combined_promoter_delta_variants_bootstrap.RData')
quantile(combined_promoter_delta_variants_bootstrap, c(0.05, 0.95))
load('RData/combined_promoter_delta_variants_bootstrap.RData')

# # ALL promoter variants const pG4 bootstrap
combined_promoter_const_variants_bootstrap <- chunked_bootstrap(const_all_promoter_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_const_variants_bootstrap, file = 'RData/combined_promoter_const_variants_bootstrap.RData')
quantile(combined_promoter_const_variants_bootstrap, c(0.05, 0.95))
load("RData/combined_promoter_const_variants_bootstrap.RData")

# ALL promoter variants stable pG4 bootstrap
combined_promoter_stable_variants_bootstrap <- chunked_bootstrap(all_promoter_stable_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_stable_variants_bootstrap, file = 'RData/combined_promoter_stable_variants_bootstrap.RData')
quantile(combined_promoter_stable_variants_bootstrap, c(0.05, 0.95))
load('RData/combined_promoter_stable_variants_bootstrap.RData')

# ALL promoter variants unstable pG4 bootstrap
combined_promoter_unstable_variants_bootstrap <- chunked_bootstrap(all_promoter_unstable_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_unstable_variants_bootstrap, file = 'RData/combined_promoter_unstable_variants_bootstrap.RData')
quantile(combined_promoter_unstable_variants_bootstrap, c(0.05, 0.95))
# load('RData/combined_promoter_unstable_variants_bootstrap.RData')

# # All promoter variants plof bootstrap
all_promoter_plof_bootstrap <- chunked_bootstrap(all_promoter_plof, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_plof_bootstrap, file = 'RData/all_promoter_plof_bootstrap.RData')
quantile(all_promoter_plof_bootstrap, c(0.05, 0.95))
load('RData/all_promoter_plof_bootstrap.RData')

 # ALL promoter variants disease genes bootstrap
all_promoter_clinvar_dx_bootstrap <- chunked_bootstrap(all_promoter_clinvar_signal, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_clinvar_dx_bootstrap, file = 'RData/all_promoter_clinvar_dx_bootstrap.RData')
quantile(all_promoter_clinvar_dx_bootstrap, c(0.05, 0.95))
# load('RData/all_promoter_clinvar_dx_bootstrap.RData')

# # ALL promoter variants disease genes bootstrap
all_promoter_cgcs_dx_bootstrap <- chunked_bootstrap(all_promoter_cgcs_signal, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_cgcs_dx_bootstrap, file = 'RData/all_promoter_cgcs_dx_bootstrap.RData')
quantile(all_promoter_cgcs_dx_bootstrap, c(0.05, 0.95))
# load('RData/all_promoter_cgcs_dx_bootstrap.RData')

# # ALL promoter variants disease genes bootstrap
all_promoter_tsgs_dx_bootstrap <- chunked_bootstrap(all_promoter_tsgs_signal, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_tsgs_dx_bootstrap, file = 'RData/all_promoter_tsgs_dx_bootstrap.RData')
quantile(all_promoter_tsgs_dx_bootstrap, c(0.05, 0.95))
# load('RData/all_promoter_tsgs_dx_bootstrap.RData')
                                      
                     
confidence_intervals <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                             'pG4 G1' = g4_combined_bootstraps$G1,
                             'pG4 G2' = g4_combined_bootstraps$G2,
                             'pG4 G3' = g4_combined_bootstraps$G3,
                             'Top-third SNV' = all_promoter_plof_bootstrap,
                             'Top-third pG4 G1' = combined_g4_plof_gruns_bootstrap$G1,
                             'Top-third pG4 G2' = combined_g4_plof_gruns_bootstrap$G2,
                             'Top-third pG4 G3' = combined_g4_plof_gruns_bootstrap$G3,
                             'ClinVar SNV' = all_promoter_clinvar_dx_bootstrap,
                             'ClinVar pG4 G1' = combined_g4_clinvar_signal_bootstrap$G1,
                             'ClinVar pG4 G2' = combined_g4_clinvar_signal_bootstrap$G2,
                             'ClinVar pG4 G3' = combined_g4_clinvar_signal_bootstrap$G3,
                             'Cosmic SNV' = all_promoter_cgcs_dx_bootstrap,
                             'Cosmic pG4 G1' = combined_g4_cgcs_signal_bootstrap$G1,
                             'Cosmic pG4 G2' = combined_g4_cgcs_signal_bootstrap$G2,
                             'Cosmic pG4 G3' = combined_g4_cgcs_signal_bootstrap$G3,

                             'TsGenes SNV' = all_promoter_tsgs_dx_bootstrap,
                             'TsGenes pG4 G1' = combined_g4_tsgs_signal_bootstrap$G1,
                             'TsGenes pG4 G2' = combined_g4_tsgs_signal_bootstrap$G2,
                             'TsGenes pG4 G3' = combined_g4_tsgs_signal_bootstrap$G3
                             )

confidence_intervals <- sapply(confidence_intervals, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()

confidence_intervals$MAPS <- c(maps_all_promoter$all['MAPS'],
                               maps_pg4_combined$G1['MAPS'],
                               maps_pg4_combined$G2['MAPS'],
                               maps_pg4_combined$G3['MAPS'],
                               maps_plof_all_variants$all_promoter_plof['MAPS'],
                               maps_plof_gruns$G1['MAPS'],
                               maps_plof_gruns$G2['MAPS'],
                               maps_plof_gruns$G3['MAPS'],
                               maps_clinvar_signal_all_variants$clinvar['MAPS'],
                               maps_clinvar_pg4$G1['MAPS'],
                               maps_clinvar_pg4$G2['MAPS'],
                               maps_clinvar_pg4$G3['MAPS'],
                               maps_cgcs_signal_all_variants$cgcs["MAPS"],
                               maps_cgcs_pg4$G1['MAPS'],
                               maps_cgcs_pg4$G2["MAPS"],
                               maps_cgcs_pg4$G3["MAPS"],
                               maps_tsgs_signal_all_variants$tsgs["MAPS"],
                               maps_tsgs_pg4$G1["MAPS"],
                               maps_tsgs_pg4$G2["MAPS"],
                               maps_tsgs_pg4$G3["MAPS"]
                               )

plot_df <- confidence_intervals
plot_df$Category <- factor(c("All", "All", "All", "All",
                             "LOEUF Top 1/3", "LOEUF Top 1/3", "LOEUF Top 1/3", "LOEUF Top 1/3",
                             "ClinVar Genes", "ClinVar Genes", "ClinVar Genes", "ClinVar Genes",
                             "Cancer Gene Census","Cancer Gene Census","Cancer Gene Census","Cancer Gene Census",
                             "Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene"
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               length(all_promoter_plof),
               sapply(combined_g4_plof_gruns, function(x) length(x)),
               length(all_promoter_clinvar_signal),
               sapply(combined_g4_clinvar_signal_gruns, function(x) length(x)),
               length(all_promoter_cgcs_signal),
               sapply(combined_g4_cgcs_signal_gruns, function(x) length(x)),
               length(all_promoter_tsgs_signal),
               sapply(combined_g4_tsgs_signal_gruns, function(x) length(x))
)
plot_df$Context <- paste0(rep(c('All SNV','pG4 G1', 'pG4 G2', 'pG4 G3'), 5),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4'),5)

# tiff('figures/MAPS.tiff', units = 'in', width = 9, height = 6, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0)) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
#  geom_segment(x = 10, xend = 10, y = plot_df['ClinVar pG4 G1',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
#  geom_segment(x = 8, xend = 8, y = plot_df['Top-third pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
  geom_point() +
  ylim(c(-0.05, 0.10)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'red') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 1, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
        axis.title.y = element_text(),
        plot.margin=unit(c(0,0,0,0),"cm")) +
  annotate("text", x = 1.2, y = 0.043, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 1.4, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 1.0, y = 0.096, label = "pLOF", color = 'red', size = 2.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 3) +
  annotate("text", x = 6.5, y = -0.05, label = 'LOEUF Top 1/3', size = 3) +
  annotate("text", x = 10.5, y = -0.05, label = 'Disease Genes', size = 3)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 3)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor suppressor gene', size = 3)
ggsave('figures/MAPS.tiff', units = 'in', width = 9, height = 6, device=tiff())
ggsave('figures/MAPS.pdf', width = 9, height = 6,device=pdf())
dev.off()

###################################
## Plot delta pG4 vs. const. pG4 ##
###################################

confidence_intervals_2 <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                               'pG4 G1' = g4_combined_bootstraps$G1,
                               'pG4 G2' = g4_combined_bootstraps$G2,
                               'pG4 G3' = g4_combined_bootstraps$G3,
                               'Delta SNV' = combined_promoter_delta_variants_bootstrap,
                               'Delta pG4 G1' = g4_combined_delta_bootstraps$G1,
                               'Delta pG4 G2' = g4_combined_delta_bootstraps$G2,
                               'Delta pG4 G3' = g4_combined_delta_bootstraps$G3,
                               'ClinVar SNV' = all_promoter_clinvar_dx_bootstrap,
                               'ClinVar pG4 G1' = combined_g4_clinvar_signal_bootstrap$G1,
                               'ClinVar pG4 G2' = combined_g4_clinvar_signal_bootstrap$G2,
                               'ClinVar pG4 G3' = combined_g4_clinvar_signal_bootstrap$G3,
                               'Cosmic SNV' = all_promoter_cgcs_dx_bootstrap,
                               'Cosmic pG4 G1' = combined_g4_cgcs_signal_bootstrap$G1,
                               'Cosmic pG4 G2' = combined_g4_cgcs_signal_bootstrap$G2,
                               'Cosmic pG4 G3' = combined_g4_cgcs_signal_bootstrap$G3,
                               'TsGenes SNV' = all_promoter_tsgs_dx_bootstrap,
                               'TsGenes pG4 G1' = combined_g4_tsgs_signal_bootstrap$G1,
                               'TsGenes pG4 G2' = combined_g4_tsgs_signal_bootstrap$G2,
                               'TsGenes pG4 G3' = combined_g4_tsgs_signal_bootstrap$G3
                             )

confidence_intervals_2 <- sapply(confidence_intervals_2, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()
confidence_intervals_2$MAPS <- c(maps_all_promoter$all['MAPS'],
                                 maps_pg4_combined$G1['MAPS'],
                                 maps_pg4_combined$G2['MAPS'],
                                 maps_pg4_combined$G3['MAPS'],
                                 maps_all_promoter_delta$all['MAPS'],
                                 maps_promoter_combined_delta$G1['MAPS'],
                                 maps_promoter_combined_delta$G2['MAPS'],
                                 maps_promoter_combined_delta$G3['MAPS'],
                                 maps_clinvar_signal_all_variants$clinvar['MAPS'],
                                 maps_clinvar_pg4$G1['MAPS'],
                                 maps_clinvar_pg4$G2['MAPS'],
                                 maps_clinvar_pg4$G3['MAPS'],
                                 maps_cgcs_signal_all_variants$cgcs["MAPS"],
                                 maps_cgcs_pg4$G1['MAPS'],
                                 maps_cgcs_pg4$G2["MAPS"],
                                 maps_cgcs_pg4$G3["MAPS"],
                                 maps_tsgs_signal_all_variants$tsgs["MAPS"],
                                 maps_tsgs_pg4$G1["MAPS"],
                                 maps_tsgs_pg4$G2["MAPS"],
                                 maps_tsgs_pg4$G3["MAPS"]
                               )

plot_df <- confidence_intervals_2
plot_df$Category <- factor(c("All", "All", "All", "All",
                             "Alternative", "Alternative", "Alternative", "Alternative",
                             "ClinVar", "ClinVar", "ClinVar", "ClinVar",
                             "Cancer Gene Census","Cancer Gene Census","Cancer Gene Census","Cancer Gene Census",
                             "Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene"
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               length(all_promoter_delta_variants),
               sapply(promoter_combined_delta, function(x) length(x)),
               length(all_promoter_clinvar_signal),
               sapply(combined_g4_clinvar_signal_gruns, function(x) length(x)),
               length(all_promoter_cgcs_signal),
               sapply(combined_g4_cgcs_signal_gruns, function(x) length(x)),
               length(all_promoter_tsgs_signal),
               sapply(combined_g4_tsgs_signal_gruns, function(x) length(x))
)

plot_df$Context <- paste0(rep(c('All SNV','pG4 G1', 'pG4 G2', 'pG4 G3'), 5),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4'),5)

# tiff('figures/MAPS_delta_dx.tiff', units = 'in', width = 9, height = 6, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0)) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
  geom_point() +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 1, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
        axis.title.y = element_text(),
        plot.margin=unit(c(0,0,0,0),"cm")) +
  annotate("text", x = 1, y = 0.043, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 1.2, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 0.8, y = 0.096, label = "pLOF", color = 'purple', size = 2.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 3) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 3) +
  annotate("text", x = 10.5, y = -0.05, label = 'Diseae Genes', size = 3)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 3)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor suppressor gene', size = 3)

ggsave('figures/MAPS_delta_dx.tiff', units = 'in', width = 9, height = 6,device=tiff())
ggsave('figures/MAPS_delta_dx.pdf', width = 9, height = 6, device=pdf())
dev.off()

tiff('figures/MAPS_delta_dx_resized.tiff', units = 'in', width = 6, height = 4, res = 600)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0), size = 0.25) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.2) +
  geom_point(size = 0.5) +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 0.5, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 4),
        axis.text.y = element_text(hjust = 1, size = 7),
        axis.title.y = element_text(size=5),
        plot.margin=unit(c(0,0,0,0),"cm"),
        legend.title = element_text(size=5),
        legend.text = element_text(size=5)) +
  annotate("text", x = 1.3, y = 0.043, label = "missense", color = 'orange', size = 1.5) +
  annotate("text", x = 1.55, y = 0.005, label = "synonymous", color = 'grey', size = 1.5) +
  annotate("text", x = 1, y = 0.096, label = "pLOF", color = 'purple', size = 1.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 2) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 2) +
  annotate("text", x = 10.5, y = -0.05, label = 'Disease Genes', size = 2)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 2)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor suppressor gene', size = 2)
dev.off()

# pdf('figures/MAPS_delta_dx_resized.pdf', width = 6, height = 4, res = 600)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0), size = 0.25) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.2) +
  geom_point(size = 0.5) +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 0.5, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 5),
        axis.text.y = element_text(hjust = 1, size = 9),
        axis.title.y = element_text(size=9),
        plot.margin=unit(c(0,0,0,0),"cm"),
        legend.title = element_text(size=5),
        legend.text = element_text(size=5)) +
  annotate("text", x = 1.3, y = 0.043, label = "missense", color = 'orange', size = 1.5) +
  annotate("text", x = 1.55, y = 0.005, label = "synonymous", color = 'grey', size = 1.5) +
  annotate("text", x = 1, y = 0.096, label = "pLOF", color = 'purple', size = 1.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 2) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 2) +
  annotate("text", x = 10.5, y = -0.05, label = 'Disease Genes', size = 2)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 2)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor Suppressor Gene', size = 2)
# grDevices::embedFonts('figures/MAPS_delta_dx_resized.pdf')
ggsave('figures/MAPS_delta_dx_resized.pdf', width = 6, height = 4,device = pdf())
ggsave('figures/MAPS_delta_dx_resized.tiff', width = 6, height = 4,device = tiff())
dev.off()

###########################################
## Plot all G4, delta G4, and clinvar G4 ##
###########################################

confidence_intervals_2 <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                               'pG4 G1' = g4_combined_bootstraps$G1,
                               'pG4 G2' = g4_combined_bootstraps$G2,
                               'pG4 G3' = g4_combined_bootstraps$G3,
                               'Delta SNV' = combined_promoter_delta_variants_bootstrap,
                               'Delta pG4 G1' = g4_combined_delta_bootstraps$G1,
                               'Delta pG4 G2' = g4_combined_delta_bootstraps$G2,
                               'Delta pG4 G3' = g4_combined_delta_bootstraps$G3,
                               'Const. SNV' = combined_promoter_const_variants_bootstrap,
                               'Const. pG4 G1' = g4_combined_const_bootstraps$G1,
                               'Const. pG4 G2' = g4_combined_const_bootstraps$G2,
                               'Const. pG4 G3' = g4_combined_const_bootstraps$G3,

                               'Stable SNV' = combined_promoter_stable_variants_bootstrap,
                               'Stable pG4 G1' = g4_combined_stable_bootstraps$G1,
                               'Stable pG4 G2' = g4_combined_stable_bootstraps$G2,
                               'Stable pG4 G3' = g4_combined_stable_bootstraps$G3,

                               'Unstable SNV' = combined_promoter_unstable_variants_bootstrap,
                               'Unstable pG4 G1' = g4_combined_unstable_bootstraps$G1,
                               'Unstable pG4 G2' = g4_combined_unstable_bootstraps$G2,
                               'Unstable pG4 G3' = g4_combined_unstable_bootstraps$G3

)

confidence_intervals_2 <- sapply(confidence_intervals_2, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()

confidence_intervals_2$MAPS <- c(maps_all_promoter$all['MAPS'],
                                 maps_pg4_combined$G1['MAPS'],
                                 maps_pg4_combined$G2['MAPS'],
                                 maps_pg4_combined$G3['MAPS'],

                                 maps_all_promoter_delta$all['MAPS'],
                                 maps_promoter_combined_delta$G1['MAPS'],
                                 maps_promoter_combined_delta$G2['MAPS'],
                                 maps_promoter_combined_delta$G3['MAPS'],

                                 maps_all_promoter_const$all['MAPS'],
                                 maps_promoter_combined_const$G1['MAPS'],
                                 maps_promoter_combined_const$G2['MAPS'],
                                 maps_promoter_combined_const$G3['MAPS'],

                                 maps_all_promoter_stable$all['MAPS'],
                                 maps_promoter_combined_stable$G1['MAPS'],
                                 maps_promoter_combined_stable$G2['MAPS'],
                                 maps_promoter_combined_stable$G3['MAPS'],

                                 maps_all_promoter_unstable$all['MAPS'],
                                 maps_promoter_combined_unstable$G1['MAPS'],
                                 maps_promoter_combined_unstable$G2['MAPS'],
                                 maps_promoter_combined_unstable$G3['MAPS']

                                 )

plot_df <- confidence_intervals_2
plot_df$Category <- factor(c("All", "All", "All", "All",
                             "Alternative", "Alternative", "Alternative", "Alternative",
                             "Const.", "Const.", "Const.", "Const.",
                             "Stable.", "Stable.", "Stable.", "Stable.",
                             "Unstable.", "Unstable.", "Unstable.", "Unstable."
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               length(all_promoter_delta_variants),
               sapply(promoter_combined_delta, function(x) length(x)),
               length(const_all_promoter_variants),
               sapply(promoter_combined_const, function(x) length(x)),
               length(all_promoter_stable_variants),
               sapply(promoter_combined_stable, function(x) length(x)),

               length(all_promoter_unstable_variants),
               sapply(promoter_combined_unstable, function(x) length(x))
               )

plot_df$Context <- paste0(rep(c('All SNV','G4 G1', 'G4 G2', 'G4 G3'), 5),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4'),5)
# tiff('figures/MAPS_delta_const.tiff', units = 'in', width = 7, height = 5, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0)) +
  geom_point() +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 1, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
        axis.title.y = element_text(),
        plot.margin=unit(c(0,0,0,0),"cm")) +
  annotate("text", x = 1, y = 0.044, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 1.2, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 0.8, y = 0.096, label = "pLOF", color = 'purple', size = 2.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All pG4s', size = 3) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 3) +
  annotate("text", x = 10.5, y = -0.05, label = 'Cons. pG4', size = 3) +
  annotate("text", x = 14.5, y = -0.05, label = 'Stable. pG4s', size = 3) +
  annotate("text", x = 18.5, y = -0.05, label = 'Unstable. pG4', size = 3)
ggsave('figures/MAPS_delta_const.tiff', units = 'in', width = 12, height = 5, dpi = 300,device=tiff())
ggsave('figures/MAPS_delta_const.pdf', width = 12, height = 5, device=pdf())
dev.off()


###########################################
## Plot with comparison of non-pG4 Gruns ##
###########################################

confidence_intervals_2 <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                               'pG4 G1' = g4_combined_bootstraps$G1,
                               'pG4 G2' = g4_combined_bootstraps$G2,
                               'pG4 G3' = g4_combined_bootstraps$G3,
                               'non-G4 1' = nog4_combined_bootstraps$pos1,
                               'non-G4 2' = nog4_combined_bootstraps$pos2,
                               'non-G4 3' = nog4_combined_bootstraps$pos3,
                               'plof All SNV' = all_promoter_plof_bootstrap,
                               'plof pG4 G1' = combined_g4_plof_gruns_bootstrap$G1,
                               'plof pG4 G2' = combined_g4_plof_gruns_bootstrap$G2,
                               'plof pG4 G3' = combined_g4_plof_gruns_bootstrap$G3,
                               'plof non-G4 1' = combined_npg4_plof_gruns_bootstrap$pos1,
                               'plof non-G4 2' = combined_npg4_plof_gruns_bootstrap$pos2,
                               'plof non-G4 3' = combined_npg4_plof_gruns_bootstrap$pos3,
                               'Delta SNV' = combined_promoter_delta_variants_bootstrap,
                               'Delta pG4 G1' = g4_combined_delta_bootstraps$G1,
                               'Delta pG4 G2' = g4_combined_delta_bootstraps$G2,
                               'Delta pG4 G3' = g4_combined_delta_bootstraps$G3,
                               'Delta non-G4 1' = nog4_combined_delta_bootstraps$pos1,
                               'Delta non-G4 2' = nog4_combined_delta_bootstraps$pos2,
                               'Delta non-G4 3' = nog4_combined_delta_bootstraps$pos3,
                               'Const. SNV' = combined_promoter_const_variants_bootstrap,
                               'Const. pG4 G1' = g4_combined_const_bootstraps$G1,
                               'Const. pG4 G2' = g4_combined_const_bootstraps$G2,
                               'Const. pG4 G3' = g4_combined_const_bootstraps$G3,
                               'Const. non-G4 1' = nog4_combined_const_bootstraps$pos1,
                               'Const. non-G4 2' = nog4_combined_const_bootstraps$pos2,
                               'Const. non-G4 3' = nog4_combined_const_bootstraps$pos3,

                               'Stable SNV' = combined_promoter_stable_variants_bootstrap,
                               'Stable pG4 G1' = g4_combined_stable_bootstraps$G1,
                               'Stable pG4 G2' = g4_combined_stable_bootstraps$G2,
                               'Stable pG4 G3' = g4_combined_stable_bootstraps$G3,
                               'Stable non-G4 1' = nog4_combined_stable_bootstraps$pos1,
                               'Stable non-G4 2' = nog4_combined_stable_bootstraps$pos2,
                               'Stable non-G4 3' = nog4_combined_stable_bootstraps$pos3,


                               'Unstable SNV' = combined_promoter_unstable_variants_bootstrap,
                               'Unstable pG4 G1' = g4_combined_unstable_bootstraps$G1,
                               'Unstable pG4 G2' = g4_combined_unstable_bootstraps$G2,
                               'Unstable pG4 G3' = g4_combined_unstable_bootstraps$G3,
                               'Unstable non-G4 1' = nog4_combined_unstable_bootstraps$pos1,
                               'Unstable non-G4 2' = nog4_combined_unstable_bootstraps$pos2,
                               'Unstable non-G4 3' = nog4_combined_unstable_bootstraps$pos3,

                               'ClinVar SNV' = all_promoter_clinvar_dx_bootstrap,
                               'ClinVar pG4 G1' = combined_g4_clinvar_signal_bootstrap$G1,
                               'ClinVar pG4 G2' = combined_g4_clinvar_signal_bootstrap$G2,
                               'ClinVar pG4 G3' = combined_g4_clinvar_signal_bootstrap$G3,
                               'ClinVar non-G4 1' = nog4_combined_clinvar_signal_bootstrap$pos1,
                               'ClinVar non-G4 2' = nog4_combined_clinvar_signal_bootstrap$pos2,
                               'ClinVar non-G4 3' = nog4_combined_clinvar_signal_bootstrap$pos3,

                               'Cosmic SNV' = all_promoter_cgcs_dx_bootstrap,
                               'Cosmic pG4 G1' = combined_g4_cgcs_signal_bootstrap$G1,
                               'Cosmic pG4 G2' = combined_g4_cgcs_signal_bootstrap$G2,
                               'Cosmic pG4 G3' = combined_g4_cgcs_signal_bootstrap$G3,
                               'Cosmic non-G4 1' = nog4_combined_cgcs_signal_bootstrap$pos1,
                               'Cosmic non-G4 2' = nog4_combined_cgcs_signal_bootstrap$pos2,
                               'Cosmic non-G4 3' = nog4_combined_cgcs_signal_bootstrap$pos3,

                               'TsGenes SNV' = all_promoter_tsgs_dx_bootstrap,
                               'TsGenes pG4 G1' = combined_g4_tsgs_signal_bootstrap$G1,
                               'TsGenes pG4 G2' = combined_g4_tsgs_signal_bootstrap$G2,
                               'TsGenes pG4 G3' = combined_g4_tsgs_signal_bootstrap$G3,
                               'TsGenes non-G4 1' = nog4_combined_tsgs_signal_bootstrap$pos1,
                               'TsGenes non-G4 2' = nog4_combined_tsgs_signal_bootstrap$pos2,
                               'TsGenes non-G4 3' = nog4_combined_tsgs_signal_bootstrap$pos3
)
confidence_intervals_2 <- sapply(confidence_intervals_2, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()

confidence_intervals_2$MAPS <- c(maps_all_promoter$all['MAPS'],
                                 maps_pg4_combined$G1['MAPS'],
                                 maps_pg4_combined$G2['MAPS'],
                                 maps_pg4_combined$G3['MAPS'],
                                 maps_nong4_first_last$pos1['MAPS'],
                                 maps_nong4_first_last$pos2['MAPS'],
                                 maps_nong4_first_last$pos3['MAPS'],

                                 maps_plof_all_variants$all_promoter_plof['MAPS'],
                                 maps_plof_gruns$G1['MAPS'],
                                 maps_plof_gruns$G2['MAPS'],
                                 maps_plof_gruns$G3['MAPS'],
                                 maps_nog4_combined_plof$pos1['MAPS'],
                                 maps_nog4_combined_plof$pos2['MAPS'],
                                 maps_nog4_combined_plof$pos3['MAPS'],

                                 maps_all_promoter_delta$all['MAPS'],
                                 maps_promoter_combined_delta$G1['MAPS'],
                                 maps_promoter_combined_delta$G2['MAPS'],
                                 maps_promoter_combined_delta$G3['MAPS'],
                                 maps_promoter_nog4_delta$pos1['MAPS'],
                                 maps_promoter_nog4_delta$pos2['MAPS'],
                                 maps_promoter_nog4_delta$pos3['MAPS'],

                                 maps_all_promoter_const$all['MAPS'],
                                 maps_promoter_combined_const$G1['MAPS'],
                                 maps_promoter_combined_const$G2['MAPS'],
                                 maps_promoter_combined_const$G3['MAPS'],
                                 maps_promoter_nog4_const$pos1['MAPS'],
                                 maps_promoter_nog4_const$pos2['MAPS'],
                                 maps_promoter_nog4_const$pos3['MAPS'],

                                 maps_all_promoter_stable$all['MAPS'],
                                 maps_promoter_combined_stable$G1['MAPS'],
                                 maps_promoter_combined_stable$G2['MAPS'],
                                 maps_promoter_combined_stable$G3['MAPS'],
                                 maps_promoter_nog4_stable$pos1['MAPS'],
                                 maps_promoter_nog4_stable$pos2['MAPS'],
                                 maps_promoter_nog4_stable$pos3['MAPS'],

                                 maps_all_promoter_unstable$all['MAPS'],
                                 maps_promoter_combined_unstable$G1['MAPS'],
                                 maps_promoter_combined_unstable$G2['MAPS'],
                                 maps_promoter_combined_unstable$G3['MAPS'],
                                 maps_promoter_nog4_unstable$pos1['MAPS'],
                                 maps_promoter_nog4_unstable$pos2['MAPS'],
                                 maps_promoter_nog4_unstable$pos3['MAPS'],

                                 maps_clinvar_signal_all_variants$clinvar['MAPS'],
                                 maps_clinvar_pg4$G1['MAPS'],
                                 maps_clinvar_pg4$G2['MAPS'],
                                 maps_clinvar_pg4$G3['MAPS'],
                                 maps_clinvar_nong4$pos1['MAPS'],
                                 maps_clinvar_nong4$pos2['MAPS'],
                                 maps_clinvar_nong4$pos3['MAPS'],

                                 maps_cgcs_signal_all_variants$cgcs['MAPS'],
                                 maps_cgcs_pg4$G1['MAPS'],
                                 maps_cgcs_pg4$G2['MAPS'],
                                 maps_cgcs_pg4$G3['MAPS'],
                                 maps_cgcs_nong4$pos1['MAPS'],
                                 maps_cgcs_nong4$pos2['MAPS'],
                                 maps_cgcs_nong4$pos3['MAPS'],

                                 maps_tsgs_signal_all_variants$tsgs['MAPS'],
                                 maps_tsgs_pg4$G1['MAPS'],
                                 maps_tsgs_pg4$G2['MAPS'],
                                 maps_tsgs_pg4$G3['MAPS'],
                                 maps_tsgs_nong4$pos1['MAPS'],
                                 maps_tsgs_nong4$pos2['MAPS'],
                                 maps_tsgs_nong4$pos3['MAPS']
)
plot_df <- confidence_intervals_2
plot_df$Category <- factor(c(rep("All", 7),
                             rep('Top 1/3 LOEUF', 7),
                             rep("Alternative", 7),
                             rep('Constitutive', 7),
                             rep("Stable", 7),
                             rep('Unstable', 7),
                             rep('ClinVar', 7),
                             rep('Cancer Gene Cencus',7),
                             rep('Tumor Suppressor Genes',7)
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               sapply(promoter_nog4_combined, function(x) length(x)),

               length(all_promoter_plof),
               sapply(combined_g4_plof_gruns, function(x) length(x)),
               sapply(nog4_combined_plof, function(x) length(x)),

               length(all_promoter_delta_variants),
               sapply(promoter_combined_delta, function(x) length(x)),
               sapply(promoter_nog4_combined_delta, function(x) length(x)),

               length(const_all_promoter_variants),
               sapply(promoter_combined_const, function(x) length(x)),
               sapply(promoter_nog4_combined_const, function(x) length(x)),

               length(all_promoter_stable_variants),
               sapply(promoter_combined_stable, function(x) length(x)),
               sapply(promoter_nog4_combined_stable, function(x) length(x)),

               length(all_promoter_unstable_variants),
               sapply(promoter_combined_unstable, function(x) length(x)),
               sapply(promoter_nog4_combined_unstable, function(x) length(x)),

               length(all_promoter_clinvar_signal),
               sapply(combined_g4_clinvar_signal_delta_gruns, function(x) length(x)),
               sapply(nog4_combined_clinvar_signal, function(x) length(x)),

               length(all_promoter_cgcs_signal),
               sapply(combined_g4_cgcs_signal_delta_gruns, function(x) length(x)),
               sapply(nog4_combined_cgcs_signal, function(x) length(x)),

               length(all_promoter_tsgs_signal),
               sapply(combined_g4_tsgs_signal_delta_gruns, function(x) length(x)),
               sapply(nog4_combined_tsgs_signal, function(x) length(x))
               )
plot_df$Context <- paste0(rep(c('All SNV','pG4 G1', 'pG4 G2', 'pG4 G3', 'non-G4 1', 'non-G4 2', 'non-G4 3'), 9),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4', 'non-G4', 'non-G4', 'non-G4'),9)

# tiff('figures/MAPS_all_classes_3.tiff', units = 'in', width = 10, height = 4, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0), size = 0.25) +
  # geom_segment(x = 32, xend = 32, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.2) +
  geom_point(size = 0.5) +
  ylim(c(-0.05, .12)) + ylab("MAPS") +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red", "blue")) +
  theme(axis.line = element_line(size = 0.5, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 65, hjust = 1, size = 5),
        axis.title.y = element_text(size = 8),
        axis.text.y = element_text(size = 8),
        plot.margin=unit(c(0,0,0,0),"cm"),
        legend.position = 'top',
        legend.title = element_blank(),
        legend.text = element_text(size=5)) +
  annotate("text", x = 2, y = 0.043, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 2, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 1.5, y = 0.096, label = "pLOF", color = 'purple', size = 2.5) +
  geom_vline(xintercept = 7.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 14.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 15 + 6.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 28.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 35.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 42.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 49.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 56.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 3.75, y = -0.05, label = 'All Promiters', size = 2) +
  annotate("text", x = 11.25, y = -0.05, label = 'Top 1/3 LOEUF', size = 2) +
  annotate("text", x = 17.75, y = -0.05, label = 'Alt. pG4', size = 2) +
  annotate("text", x = 25.25, y = -0.05, label = 'Const. pG4', size = 2) +
  annotate("text", x = 24.25 + 7.5, y = -0.05, label = 'Stable pG4', size = 2) +
  annotate("text", x = 32.25 + 7, y = -0.05, label = 'Unstable pG4', size = 2) +
  annotate("text", x = 39.25 + 7.5, y = -0.05, label = 'ClinVar pG4', size = 2)+
  annotate("text", x = 46.25 + 7, y = -0.05, label = 'Cancer Gene Cencus pG4', size = 2)+
  annotate("text", x = 54.25 + 7, y = -0.05, label = 'Tumor Suppressor Genes pG4', size = 2)
ggsave('figures/MAPS_all_classes_3.tiff', units = 'in', width = 10, height = 4, dpi = 300,device = tiff())
ggsave('figures/MAPS_all_classes_3.pdf', width = 10, height = 4, device = pdf())
dev.off()


In [ ]:
# 0.2kb_promoter_MAPS.R
library(dplyr)
library(GenomicRanges)
library(ggplot2)
#library(ensembldb)

setwd("/disk2/G4_MS/frequency/MAPS_h3k4me3")
source("/disk2/G4_MS/frequency/MAPS/data/h3k4me3_stable_unstable.data.get.R")
load("/disk2/G4_MS/frequency/MAPS/data/RData/ps_lm_genomes.RData")

gtffile <- "/disk2/G4_MS/frequency/MAPS/data/Homo_sapiens.GRCh38.104.gtf.gz"
DB.104 <- ensembldb::ensDbFromGtf(gtf = gtffile)
EnsDb.Hsapiens.v104 <- ensembldb::EnsDb(DB.104)
## cds rangeget
mu <- data.table::fread("/disk2/G4_MS/frequency/MAPS/data/mutation_rate_methylation_bins.txt")
mu <- mu %>%
  mutate(ref_context = paste0(context,":",methylation_level),
         mut_context = paste0(substr(context,1,1),
                              alt,
                              substr(context,3,3)),
         mutation = paste0(ref_context,">",mut_context))
## Genes with alternativate pG4 isoform
dlata_g4 <- unique(intersect(reference_ranges$nog4_tx$ensembl_gene_id,reference_ranges$g4_regions$ensembl_gene_id))
constitutive_g4 <-unique(setdiff(reference_ranges$g4_regions$ensembl_gene_id,reference_ranges$nog4_tx$ensembl_gene_id))

# load constraint table by gene
gene_constraints <- data.table::fread("/disk2/G4_MS/reference/gnomad.v2.1.1.lof_metrics.by_gene.txt")
non_plof_genes <- gene_constraints[order(gene_constraints$oe_lof_upper,decreasing = TRUE),]
non_plof_genes <- non_plof_genes[1:6568,]
plof_genes <- gene_constraints[order(gene_constraints$oe_lof_upper,decreasing = FALSE),]
plof_genes <- plof_genes[1:6568,]
clinvar_genes <- data.table::fread("/disk2/G4_MS/reference/clinvar_20220115.vcf_annotated_af.tsv")
clinvar_genes <- AnnotationDbi::mapIds(EnsDb.Hsapiens.v104,
                                       keys=gsub(pattern = ":.*","",clinvar_genes$GENEINFO),
                                       column = "GENEID",keytype="SYMBOL")
cgcs <- data.table::fread("/disk2/G4_MS/reference/cancer_gene_census.tsv")
cgcs <- AnnotationDbi::mapIds(EnsDb.Hsapiens.v104,keys=cgcs$GeneSymbol,column = "GENEID",keytype="SYMBOL")

tsgs <- data.table::fread("/disk2/G4_MS/reference/tsgenes.tsv")
# tsgs <- tsgs[which(tsgs$GeneType=='protein-coding'),]
tsgs <- AnnotationDbi::mapIds(EnsDb.Hsapiens.v104,keys=tsgs$GeneSymbol,column = "GENEID",keytype="SYMBOL")

# Load genome-wide methylation estimates
load('/disk2/G4_MS/frequency/MAPS/data/methylation_hg38.RData')
promoter_gnomad <- data.table::fread("/disk2/G4_MS/frequency/MAPS/data/active_promoter_region_gnomad.genomes.v3.1.sites.pass_snps.tsv")
colnames(promoter_gnomad) <- c("CHROM", "POS", "REF", "ALT", "AN", "AC", "AF")
promoter_hept <- data.table::fread("/disk2/G4_MS/frequency/MAPS/data/active_promoter_region_gnomad.genomes.v3.1.sites.pass_snps.seq", header = FALSE)
promoter_tri <- substr(promoter_hept$V2,3,5)
promoter_gnomad$context <- toupper(promoter_tri)
promoter_gnomad$mut_context <- paste0(substr(promoter_gnomad$context,1,1),
                                      promoter_gnomad$ALT,
                                      substr(promoter_gnomad$context,3,3))
promoter_gnomad$hept <- toupper(promoter_hept$V2)
promoter_gnomad <- promoter_gnomad %>%  select(CHROM, POS, REF, ALT, AN, AC, AF, context, mut_context, hept) %>%
  filter(CHROM != 'chrX' & CHROM != 'chrY', AN > 0.8*152312, AC >= 1)
rm(promoter_hept)
rm(promoter_tri)
promoter_gnomad_grange <- granges_from_vcf(promoter_gnomad)
### 
# combined_promoter_variants = promoter_gnomad_grange
g1_allowed <- c('AGG',
                'CCA',
                'CCG',
                'CCT',
                'CGG',
                'TGG')
g2_allowed <- c('CCC','GGG')
g3_allowed <- c('ACC',
                'GCC',
                'GGA',
                'GGC',
                'GGT',
                'TCC')

allowed <- list('pos1' = g1_allowed,
                'pos2' = g2_allowed,
                'pos3' = g3_allowed)

######################
## Helper Functions ##
######################

get_ps <- function(vcf_list, mu) {

  n <- lapply(vcf_list, function(x) table(x$mutation))
  ps <- lapply(vcf_list, function(x) table(x[x$AC == 1]$mutation))

  ps_res <- mapply(function(x, y) cbind(x, y[match(names(x), names(y))]), x=n, y=ps, SIMPLIFY = FALSE)
  ps_res <- lapply(ps_res, function(x) x[match(mu$mutation, rownames(x)),])

  return(ps_res)
}


get_mu_avg <- function(ps_list, mu) {

  res <- lapply(ps_list, function(x) sum((x*mu$mu_snp), na.rm = TRUE) / sum(x, na.rm = TRUE))


  return(res)

}


calc_maps <- function(raw_ps, ps_counts, predictions) {

  obs <- unlist(raw_ps) * unlist(ps_counts)
  exp <- unlist(predictions) * unlist(ps_counts)

  maps <- (obs - exp) / unlist(ps_counts)

  return(maps)

}
# MAPS 2 model:
calc_maps2 <- function(variants, mu, ps_lm) {

  variants <- data.frame(variants)

  total_variants <- dim(variants)[[1]]

  variants <- split(variants, variants$group)

  if (length(variants) == 2) {
    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)
    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)

  } else if (names(variants) == 'no_adjust') {

    methyl_adjust <- c('exp_singletons.1' = 0,
                       'obs_singletons' = 0)
    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)
  } else {
    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)
    no_adjust <- c('exp_singletons.1' = 0,
                   'obs_singletons' = 0)
  }
  combined <- methyl_adjust + no_adjust

  maps <- (combined['obs_singletons'] - combined['exp_singletons.1']) / total_variants
  raw_p <- combined['obs_singletons'] / total_variants
  se <-  sqrt(raw_p*(1-raw_p)/total_variants)
  result <- c("MAPS" = as.numeric(maps), "SE" = as.numeric(se), "N" = as.integer(total_variants))
  return(result)

}

get_obs_exp_singletons <- function(variants, ps_lm) {

  variants <- na.omit(variants)
  mu_avg <- sum(variants$n * variants$mu_snp) / sum(variants$n)
  exp_singletons <- sum(variants$n) * predict(ps_lm, data.frame('mu_snp' = mu_avg))
  obs_singletons <- sum(variants$singletons)
  return(c('exp_singletons' = exp_singletons,
           'obs_singletons' = obs_singletons))

}

do_MAPS <- function(vcf_list, mu, ps_model) {
  # determine proportion singletons
  ps <- get_ps(vcf_list, mu)
  # determine mu average for the variants + contexts
  mu_avg <- get_mu_avg(ps, mu)
  # extract raw proportion of singletons
  raw_p <- lapply(vcf_list, function(x) length(x[x$AC==1])/length(x))
  var_counts <- lapply(vcf_list, function(x) length(x))
  se <- mapply(function(p, n ) sqrt(p*(1-p)/n), raw_p, var_counts)
  # predicted ps:
  ps_predict <- lapply(mu_avg, function(x) predict(ps_model, data.frame(mu = unlist(x))))
  maps <- calc_maps(raw_p, var_counts, ps_predict)
  return(maps)

}

annotate_grun_position <- function(grun_grange, promoter_region, width = 3) {

  gruns <- grun_grange[width(grun_grange) == width]
  gruns_df <- data.frame(gruns)
  #gruns$strand <- gruns$oristrand
  gruns_df$strand <- promoter_region$oristrand[match(gruns_df$txid, promoter_region$txid)]
  # gruns_df$strand <- gruns_df$strand.1
  # gruns_df <- gruns_df[,!colnames(gruns_df) %in% c("oristrand","strand.1")]
  tile <- makeGRangesFromDataFrame(gruns_df)
  tile <- tile(tile, width = 1) %>% as.data.frame()
  pos_tiles <- tile %>% filter(strand == '+')
  neg_tiles <- tile %>% filter(strand == '-')

  pos_tiles$var_pos <- rep(x = c(1:width), dim(pos_tiles)[[1]]/width)
  neg_tiles$var_pos <- rep(x = c(1:width), dim(neg_tiles)[[1]]/width)

  tiles <- rbind(pos_tiles, neg_tiles)

  pos1 <- tiles %>% filter(var_pos == 1)
  pos2 <- tiles %>% filter(var_pos == 2)
  pos3 <- tiles %>% filter(var_pos == 3)

  res <- list("pos1" = pos1,
              "pos2" = pos2,
              "pos3" = pos3)
  res <- lapply(res, function(x) makeGRangesFromDataFrame(x, keep.extra.columns = TRUE))
  return(res)

}
match_var_to_strand <- function(var_grange_first, var_grange_last, tx_grange) {

  # this function annotates a variant as being either first or last in a trinucleotide G/C run
  # based on the strand of the UTR transcript

  pos_strand <- tx_grange[tx_grange$oristrand == '+']
  neg_strand <- tx_grange[tx_grange$oristrand == '-']
  pos_strand_first <- subsetByOverlaps(var_grange_first, pos_strand)
  pos_strand_last <- subsetByOverlaps(var_grange_last, pos_strand)

  neg_strand_first <- subsetByOverlaps(var_grange_last, neg_strand)
  neg_strand_last <- subsetByOverlaps(var_grange_first, neg_strand)

  first_vars <- c(pos_strand_first, neg_strand_first)
  last_vars <- c(pos_strand_last, neg_strand_last)

  return(list('pos1'=first_vars,
              'pos3'=last_vars))

}

calc_maps2_bootstrap <- function(variants, mu, ps_lm) {

  total_variants <- dim(variants)[[1]]

  variants <- variants[sample(1:total_variants, total_variants, replace = TRUE),]

  variants <- split(variants, variants$group)

  if (length(variants) == 2) {

    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)

    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)

  } else if (names(variants) == 'no_adjust') {

    methyl_adjust <- c('exp_singletons.1' = 0,
                       'obs_singletons' = 0)

    no_adjust <- variants$no_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$no_adjust)

  } else {

    methyl_adjust <- variants$methyl_adjust %>%
      group_by(mut) %>%
      summarise(singletons = length(which(AC == 1)),
                n = length(AN)) %>%
      mutate(ps = singletons / n) %>%
      left_join(., mu, by = c('mut' = 'mutation')) %>%
      select(mut, singletons, n, ps, methylation_level, mu_snp) %>%
      get_obs_exp_singletons(., ps_lm = ps_lm$methyl_adjust)

    no_adjust <- c('exp_singletons.1' = 0,
                   'obs_singletons' = 0)
  }
combined <- methyl_adjust + no_adjust

  maps <- (combined['obs_singletons'] - combined['exp_singletons.1']) / total_variants

  return(as.numeric(maps))

}
                
bootstrap_maps2 <- function(variants, mu, ps_lm, ntimes = 10000) {

  variants <- data.frame(variants)

  variants <- variants %>%
    select(AC, AN, group, mut)

  res <- vector("list", length = ntimes)

  res <- lapply(res, function(x) calc_maps2_bootstrap(variants, mu, ps_lm))

  return(unlist(res))

}

resample_MAPS <- function(vcf, mu, ps_model) {

  n <- table(vcf$mut_context)
  ps <- table(vcf[vcf$AC == 1]$mut_context)

  ps <- cbind(n, ps[match(names(n), names(ps))])
  ps <- ps[match(mu$mut_context, rownames(ps)),]

  mu_average <- sum(ps*mu$mu_snp, na.rm =TRUE) / sum(ps, na.rm = TRUE)

  raw_p <- length(vcf[vcf$AC==1])/length(vcf)
  var_counts <- length(vcf)
  ps_predict <- predict(ps_model, data.frame(mu = unlist(mu_average)))

  obs <- unlist(raw_p) * unlist(var_counts)
  exp <- unlist(ps_predict) * unlist(var_counts)
  maps <- (obs-exp)/unlist(var_counts)

  return(maps)

}

generate_bootstrap <- function(vcf, n, mu, ps_model) {

  x <- length(vcf)

  res <- rep(x, n)

  samples <- lapply(res, function(x) sample(x, size = x, replace = TRUE))

  maps_bootstrap <- lapply(samples, function(x) resample_MAPS(vcf[x], mu = mu, ps_model = ps_model))

  return(unlist(maps_bootstrap))

}
                
chunked_bootstrap <- function(vcf, n, mu, ps_model) {

  chunksize <- as.integer(n/10)
  chunk1 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk2 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk3 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk4 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk5 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk6 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk7 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk8 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk9 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                            mu = mu, ps_lm = ps_model)
  chunk10 <- bootstrap_maps2(variants = vcf, ntimes = chunksize,
                             mu = mu, ps_lm = ps_model)
  result <- c(chunk1, chunk2, chunk3, chunk4, chunk5,
              chunk6, chunk7, chunk8, chunk9, chunk10)
  return(result)

}
annotate_methylation <- function(variants, mutation_table, methylation) {

  variants <- left_join(variants, methylation)
  # set missing methylation values to zero
  variants$methyl_bins[is.na(variants$methyl_bins)] <- 0
  fixed_context <- ifelse(variants$context %in% mutation_table$context,
                          variants$context,
                          chartr('ATGC','TACG',variants$context))
  fixed_mut <- ifelse(variants$context %in% mutation_table$context,
                      variants$mut_context,
                      chartr('ATGC','TACG',variants$mut_context))

  # amend mutation context for MAPS score calculation
  variants$mut <- paste0(fixed_context,':',
                         variants$methyl_bins,
                         '>',
                         fixed_mut)
  variants$mut <- ifelse(variants$mut %in% mutation_table$mutation,
                         variants$mut,
                         gsub('[1|2]',
                              '0',
                              variants$mut))
  variants$context <- fixed_context
  variants$mut_context <- fixed_mut

  # annotate variants for methylation adjustment
  var_adjust <- mutation_table %>%
    filter(methylation_level != 0) %>%
    select(context) %>% unlist()
  variants$group <- ifelse((variants$context %in% var_adjust),
                           ifelse((variants$REF == 'C' & variants$ALT == 'T') | (variants$REF == 'G' & variants$ALT == 'A'),
                                  'methyl_adjust',
                                  'no_adjust'),
                           'no_adjust')

  return(variants)

}

##############
## Analysis ##
##############
# annotate methylation context for MAPS analysis
# correct gruns
reference_granges$g4_gruns <- subsetByOverlaps(reference_granges$g4_gruns, promoter_region)
## combined_promoter_variants = promoter_gnomad_grange
promoter_gnomad_grange_df <- data.frame(promoter_gnomad_grange) %>% mutate(CHROM = seqnames, POS = start)
promoter_gnomad_grange_df <- annotate_methylation(data.frame(promoter_gnomad_grange_df),
                                                  mu,
                                                  combined_methylation_estimates)
promoter_gnomad_grange <- promoter_gnomad_grange_df %>% select(-c(CHROM, POS)) %>%
  makeGRangesFromDataFrame(., keep.extra.columns = TRUE)

# subset pG4s by alternative vs. constitutive
promoter_delta_pg4 <- reference_granges$g4_gruns[reference_granges$g4_gruns$ensembl_gene_id %in% dlata_g4]
promoter_const_pg4 <- reference_granges$g4_gruns[!reference_granges$g4_gruns$ensembl_gene_id %in% dlata_g4]
                           
promoter_stable_pg4 <- reference_granges$stable_g4_gruns
# promoter_stable_pg4 <- promoter_stable_pg4[!promoter_stable_pg4$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
# promoter_unstable_pg4 <- reference_granges$g4_gruns[reference_granges$g4_gruns$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
promoter_unstable_pg4 <- reference_granges$unstable_g4_gruns
# Annotate grun positions (G1, G2, G3) and remove sequence contexts that are mismatched
promoter_g4_gruns_positional <- annotate_grun_position(reference_granges$g4_gruns,promoter_region)
promoter_g4_vars <- lapply(promoter_g4_gruns_positional, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# repeat for delta pG4
promoter_delta_pg4 <- annotate_grun_position(promoter_delta_pg4,promoter_region)
promoter_delta_pg4 <- lapply(promoter_delta_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# repeat for constitutive pG4
promoter_const_pg4 <- annotate_grun_position(promoter_const_pg4,promoter_region)
promoter_const_pg4 <- lapply(promoter_const_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))                     

# repeat for stable pG4
promoter_stable_pg4 <- annotate_grun_position(promoter_stable_pg4,promoter_region)
promoter_stable_pg4 <- lapply(promoter_stable_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# repeat for unstable pG4
promoter_unstable_pg4 <- annotate_grun_position(promoter_unstable_pg4,promoter_region)
promoter_unstable_pg4 <- lapply(promoter_unstable_pg4, function(x) subsetByOverlaps(promoter_gnomad_grange, x))

# combine variant sets and perform MAPS
promoter_combined <- c('G1' = promoter_g4_vars$pos1,
                  'G2' = promoter_g4_vars$pos2,
                  'G3' = promoter_g4_vars$pos3
)
promoter_combined_delta <- c('G1' = promoter_delta_pg4$pos1,
                             'G2' = promoter_delta_pg4$pos2,
                             'G3' = promoter_delta_pg4$pos3
)
promoter_combined_const <- c('G1' = promoter_const_pg4$pos1,
                             'G2' = promoter_const_pg4$pos2,
                             'G3' = promoter_const_pg4$pos3
)
###
promoter_combined_stable <- c('G1' = promoter_stable_pg4$pos1,
                             'G2' = promoter_stable_pg4$pos2,
                             'G3' = promoter_stable_pg4$pos3
)
promoter_combined_unstable <- c('G1' = promoter_unstable_pg4$pos1,
                             'G2' = promoter_unstable_pg4$pos2,
                             'G3' = promoter_unstable_pg4$pos3
)
# Evaluate MAPS
maps_pg4_combined <- lapply(promoter_combined, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_delta <- lapply(promoter_combined_delta, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_const<- lapply(promoter_combined_const, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_stable <- lapply(promoter_combined_stable, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_promoter_combined_unstable <- lapply(promoter_combined_unstable, function(x) calc_maps2(x, mu, ps_lm_genomes))                                       

######################################################################
## repeat for GGG/CCC repeats to control for local sequence context ##
######################################################################

promoter_combined_npg4 <- subsetByOverlaps(promoter_gnomad_grange, promoter_combined$G1, invert = TRUE)
promoter_combined_npg4 <- subsetByOverlaps(promoter_combined_npg4, promoter_combined$G2, invert = TRUE)
promoter_combined_npg4 <- subsetByOverlaps(promoter_combined_npg4, promoter_combined$G3, invert = TRUE)
# remove CDS
promoter_combined_npg4 <- subsetByOverlaps(promoter_combined_npg4, cds_ranges, invert = TRUE)
# remove CpG sites
promoter_combined_npg4_central <- grepl('[ATG]CCC[ATG]|[ATC]GGG[ATC]', substr(promoter_combined_npg4$hept, 2, 6))
promoter_combined_npg4_central <- promoter_combined_npg4[promoter_combined_npg4_central]

# Map 1st and 3rd positions for non-pG4 sequences:
promoter_region_df <- data.frame(promoter_region)
promoter_region_df$strand <- promoter_region_df$oristrand
# promoter_region_df <- promoter_region_df[,!colnames(promoter_region_df) %in% c("oristrand")]
promoter_region_df <- makeGRangesFromDataFrame(promoter_region_df,
                                               keep.extra.columns = TRUE,
                                               seqnames.field = 'seqnames',
                                               start.field = 'start',
                                               end.field = 'end')

promoter_combined_npg4_first <- grepl('[ATG]CCC[ATG]|[ATC]GGG[ATC]', substr(promoter_combined_npg4$hept, 3, 7))
promoter_combined_npg4_first <- promoter_combined_npg4[promoter_combined_npg4_first]

promoter_combined_npg4_last <- grepl('[ATG]CCC[ATG]|[ATC]GGG[ATC]', substr(promoter_combined_npg4$hept, 1, 5))
promoter_combined_npg4_last <- promoter_combined_npg4[promoter_combined_npg4_last]

first_last_npg4 <- match_var_to_strand(promoter_combined_npg4_first, promoter_combined_npg4_last,
                                       promoter_region_df)

promoter_nog4_combined <- list('pos1' = first_last_npg4$pos1,
                      'pos2' = promoter_combined_npg4_central,
                      'pos3' = first_last_npg4$pos3)
# All non-pG4 GGG / CCC contexts
maps_nong4_first_last <- lapply(promoter_nog4_combined, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_all_promoter <- list('all'=calc_maps2(promoter_gnomad_grange, mu, ps_lm_genomes))

# non-pG4 delta GGG /CCC contexts
promoter_nog4_combined_delta <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% dlata_g4]
                                                                          ))
maps_promoter_nog4_delta <- lapply(promoter_nog4_combined_delta, function(x) calc_maps2(x, mu, ps_lm_genomes))

all_promoter_delta_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% dlata_g4]
)
all_promoter_delta_variants <- subsetByOverlaps(all_promoter_delta_variants,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$g4_regions$ensembl_gene_id]
                                                )
maps_all_promoter_delta <- list('all'=calc_maps2(all_promoter_delta_variants, mu, ps_lm_genomes))
                                
# non-pG4 const GGG /CCC contexts
promoter_nog4_combined_const <- lapply(promoter_nog4_combined, function(x)
  subsetByOverlaps(x, promoter_region_df[!promoter_region_df$ensembl_gene_id %in% dlata_g4]
))
promoter_nog4_combined_const <- lapply(promoter_nog4_combined_const, function(x)
  subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$g4_regions$ensembl_gene_id]
  )
)

maps_promoter_nog4_const <- lapply(promoter_nog4_combined_const, function(x) calc_maps2(x, mu, ps_lm_genomes))
# select variants that are not in delta pG4 genes' promoter
const_all_promoter_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           promoter_region_df[!promoter_region_df$ensembl_gene_id %in% dlata_g4])
# subset these variants by those that are in pG4 genes to isolate "constitutive" pG4s
const_all_promoter_variants <- subsetByOverlaps(const_all_promoter_variants,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$g4_regions$ensembl_gene_id]
)
maps_all_promoter_const <- list('all'=calc_maps2(const_all_promoter_variants, mu, ps_lm_genomes))

# non-pG4 stable GGG/CCC contexts
promoter_nog4_combined_stable <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
                                                                          ))
promoter_nog4_combined_stable <- lapply(promoter_nog4_combined_stable, function(x) subsetByOverlaps(x, promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
                                                                          ))
maps_promoter_nog4_stable <- lapply(promoter_nog4_combined_stable, function(x) calc_maps2(x, mu, ps_lm_genomes))

all_promoter_stable_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
)
all_promoter_stable_variants <- subsetByOverlaps(all_promoter_stable_variants,
                                           promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
)
maps_all_promoter_stable <- list('all'=calc_maps2(all_promoter_stable_variants, mu, ps_lm_genomes))

# non-pG4 unstable GGG/CCC contexts
promoter_nog4_combined_unstable <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x, promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
                                                                          ))
promoter_nog4_combined_unstable <- lapply(promoter_nog4_combined_unstable, function(x) subsetByOverlaps(x, promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
                                                                          ))
maps_promoter_nog4_unstable <- lapply(promoter_nog4_combined_unstable, function(x) calc_maps2(x, mu, ps_lm_genomes))

all_promoter_unstable_variants <- subsetByOverlaps(promoter_gnomad_grange,
                                           promoter_region_df[promoter_region_df$ensembl_gene_id %in% reference_granges$unstable_g4_gruns$ensembl_gene_id]
)
all_promoter_unstable_variants <- subsetByOverlaps(all_promoter_unstable_variants,
                                           promoter_region_df[!promoter_region_df$ensembl_gene_id %in% reference_granges$stable_g4_gruns$ensembl_gene_id]
)
maps_all_promoter_unstable <- list('all'=calc_maps2(all_promoter_unstable_variants, mu, ps_lm_genomes)) 
                                      
####################################################################
## Evaluate MAPS for variants in top 1/3 of most constrained genes ##
#####################################################################
## promoter_g4_gruns = combined_g4_gruns
promoter_g4_gruns <- reference_granges$g4_gruns[width(reference_granges$g4_gruns) == 3]
promoter_g4_gruns$plof <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% plof_genes$gene_id,
                                 TRUE,
                                 FALSE)

promoter_combined_g4_plof_gruns <- lapply(promoter_combined,
                                 function(x) subsetByOverlaps(
                                   x, promoter_g4_gruns[promoter_g4_gruns$plof == TRUE])
                                 )

# overlap gnomAD variants
combined_g4_plof_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$plof == TRUE]))

#combined_g4_nonplof_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$non_plof == TRUE]))

# evaluate MAPS for top third most constrained UTR
maps_plof_gruns <-  lapply(combined_g4_plof_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))

# Evaluate all variants in top 1/3 most constrained UTR
promoter_reference_grange <- data.frame(promoter_region) %>%
  select(txid,
         ensembl_gene_id,
         start,
         end,
         oristrand,
         seqnames)
promoter_reference_grange$strand <- promoter_reference_grange$oristrand
promoter_reference_grange$top_third <- ifelse(promoter_reference_grange$ensembl_gene_id %in% plof_genes$gene_id, TRUE, FALSE)
promoter_reference_grange$bottom_third <- ifelse(promoter_reference_grange$ensembl_gene_id %in% non_plof_genes$gene_id, TRUE,FALSE)
promoter_reference_grange$clinvar_signal <- ifelse(promoter_reference_grange$ensembl_gene_id %in% clinvar_genes, TRUE, FALSE)
promoter_reference_grange$cgcs_signal <- ifelse(promoter_reference_grange$ensembl_gene_id %in% cgcs, TRUE, FALSE)
promoter_reference_grange$tsgs_signal <- ifelse(promoter_reference_grange$ensembl_gene_id %in% tsgs, TRUE, FALSE)
promoter_reference_grange <- promoter_reference_grange %>% makeGRangesFromDataFrame(start.field = 'start',
                                                                            end.field = 'end',
                                                                            seqnames.field = 'seqnames', keep.extra.columns =TRUE)

# define variant subsets
all_promoter_plof <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$top_third == TRUE])
all_promoter_nonplof <- subsetByOverlaps(promoter_gnomad_grange,promoter_reference_grange[promoter_reference_grange$bottom_third == TRUE])
all_promoter_clinvar_signal <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$clinvar_signal == TRUE])
all_promoter_cgcs_signal <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$cgcs_signal == TRUE])
all_promoter_tsgs_signal <- subsetByOverlaps(promoter_gnomad_grange, promoter_reference_grange[promoter_reference_grange$tsgs_signal == TRUE])

nog4_combined_plof <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$top_third == TRUE]))
nog4_combined_clinvar_signal <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$clinvar_signal == TRUE]))
nog4_combined_cgcs_signal <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$cgcs_signal == TRUE]))
nog4_combined_tsgs_signal <- lapply(promoter_nog4_combined, function(x) subsetByOverlaps(x,promoter_reference_grange[promoter_reference_grange$tsgs_signal == TRUE]))


promoter_g4_gruns$clinvar_signal <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% clinvar_genes, TRUE, FALSE)
promoter_g4_gruns$cgcs_signal <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% cgcs, TRUE, FALSE)
promoter_g4_gruns$tsgs_signal <- ifelse(promoter_g4_gruns$ensembl_gene_id %in% tsgs, TRUE, FALSE)

#####################################
## evaluate MAPS for ClinVar genes ##
#####################################
maps_plof_all_variants <- list('all_promoter_plof' = calc_maps2(all_promoter_plof, mu, ps_lm_genomes))

maps_nog4_combined_plof <- lapply(nog4_combined_plof, function(x) calc_maps2(x, mu, ps_lm_genomes))

maps_clinvar_signal_all_variants <- list('clinvar' = calc_maps2(all_promoter_clinvar_signal, mu, ps_lm_genomes))
maps_cgcs_signal_all_variants <- list('cgcs' = calc_maps2(all_promoter_cgcs_signal, mu, ps_lm_genomes))
maps_tsgs_signal_all_variants <- list('tsgs' = calc_maps2(all_promoter_tsgs_signal, mu, ps_lm_genomes))

combined_g4_clinvar_signal_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$clinvar_signal == TRUE]))
combined_g4_cgcs_signal_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$cgcs_signal == TRUE]))
combined_g4_tsgs_signal_gruns <- lapply(promoter_combined, function(x) subsetByOverlaps(x, promoter_g4_gruns[promoter_g4_gruns$tsgs_signal == TRUE]))

maps_clinvar_pg4 <- lapply(combined_g4_clinvar_signal_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_cgcs_pg4 <- lapply(combined_g4_cgcs_signal_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_tsgs_pg4 <- lapply(combined_g4_tsgs_signal_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))

maps_clinvar_nong4 <- lapply(nog4_combined_clinvar_signal, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_cgcs_nong4 <- lapply(nog4_combined_cgcs_signal, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_tsgs_nong4 <- lapply(nog4_combined_tsgs_signal, function(x) calc_maps2(x, mu, ps_lm_genomes))


combined_g4_clinvar_signal_delta_gruns <- mapply(function(x,y) subsetByOverlaps(x,y),
                                         x = combined_g4_clinvar_signal_gruns,
                                         y = promoter_combined_delta)
combined_g4_cgcs_signal_delta_gruns <- mapply(function(x,y) subsetByOverlaps(x,y),
                                         x = combined_g4_cgcs_signal_gruns,
                                         y = promoter_combined_delta)
combined_g4_tsgs_signal_delta_gruns <- mapply(function(x,y) subsetByOverlaps(x,y),
                                         x = combined_g4_tsgs_signal_gruns,
                                         y = promoter_combined_delta)


maps_clinvar_pg4_delta <- lapply(combined_g4_clinvar_signal_delta_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_cgcs_pg4_delta <- lapply(combined_g4_cgcs_signal_delta_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))
maps_tsgs_pg4_delta <- lapply(combined_g4_tsgs_signal_delta_gruns, function(x) calc_maps2(x, mu, ps_lm_genomes))

# Generate MAPS estimates for missense and putative loss of function-causing variants in gnomAD
missense <- data.table::fread('/disk2/G4_MS/frequency/MAPS/data/gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.tsv',header=FALSE)
colnames(missense) <- c('CHROM','POS','REF','ALT','AN','AC','AF','type','other', 'vep')
heptamers <- data.table::fread("/disk2/G4_MS/frequency/MAPS/data/gnomad.genomes.v3.1.sites.missense.pass_snps.filtered.seq", header = FALSE)
missense$heptamers <- toupper(heptamers$V2)
missense$context <- toupper(substr(heptamers$V2, 3, 5))

rm(heptamers)
                              
an_adj <- 0.8*152312
missense <- missense %>% filter(AC != 0,
                                AN >= an_adj, CHROM != "chrX" & CHROM != "chrY") %>%
                        select(-c(type, other, vep))

missense$mut_context <- paste0(substr(missense$context, 1, 1),
                               missense$ALT,
                               substr(missense$context, 3, 3))

missense$CHROM <- as.character(missense$CHROM)
missense <- annotate_methylation(missense, mu, combined_methylation_estimates)
plof <- data.table::fread("/disk2/G4_MS/frequency/MAPS/data/gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv",
                          header=FALSE,
                          col.names = c('CHROM','POS','REF','ALT','AN','AC','AF','type','other', 'vep')
)
heptamers <- data.table::fread('/disk2/G4_MS/frequency/MAPS/data/gnomad.genomes.v3.1.sites.pass_snps.high_plof.tsv.seq', header = FALSE)
plof$heptamers <- heptamers$V2 %>% toupper()
rm(heptamers)
plof <- plof %>% filter(AC >= 1,
                        AN >= 0.8*152312, CHROM != "chrX" & CHROM != "chrY",
                        nchar(REF) == 1, nchar(ALT) == 1) %>%
  select(-c(type, other, vep))

plof$context <- substr(plof$heptamers, 3,5)

plof$mut_context <- paste0(substr(plof$context, 1, 1),
                           plof$ALT,
                           substr(plof$context, 3, 3))

plof$CHROM <- as.character(plof$CHROM)
plof <- annotate_methylation(plof, mu, combined_methylation_estimates)
####
cds_varsets <- list('missense_all' = missense,
                    'lof_all' = plof)
maps_cds_markers <- lapply(cds_varsets, function(x) calc_maps2(x, mu, ps_lm_genomes))
##############################
## Generate bootstrapped CI ##
##############################
# G4_combined bootstraps
g4_combined_bootstraps <- lapply(promoter_combined, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))
save(g4_combined_bootstraps, file = 'RData/g4_positional_maps_bootstraps_ncpg.RData')
lapply(g4_combined_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/g4_positional_maps_bootstraps_ncpg.RData')

# # G4 delta bootstraps
g4_combined_delta_bootstraps <- lapply(promoter_combined_delta, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000)) #lapply(utr_combined_delta, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(g4_combined_delta_bootstraps, file = 'RData/g4_combined_delta_bootstraps.RData')
lapply(g4_combined_delta_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load("RData/g4_combined_delta_bootstraps.RData")

# # G4 const bootstraps
g4_combined_const_bootstraps <- lapply(promoter_combined_const, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(utr_combined_const, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(g4_combined_const_bootstraps, file = 'RData/g4_combined_const_bootstraps.RData')
lapply(g4_combined_const_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load("RData/g4_combined_const_bootstraps.RData")
       
# G4 stable bootstraps
#g4_combined_stable_bootstraps <- lapply(promoter_combined_stable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))
#save(g4_combined_stable_bootstraps, file = 'RData/g4_combined_stable_bootstraps.RData')
#lapply(g4_combined_stable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
load("RData/g4_combined_stable_bootstraps.RData")

# G4 unstable bootstraps
g4_combined_unstable_bootstraps <- lapply(promoter_combined_unstable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))
save(g4_combined_unstable_bootstraps, file = 'RData/g4_combined_unstable_bootstraps.RData')
lapply(g4_combined_unstable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load("RData/g4_combined_unstable_bootstraps.RData")

# # non-G4 G bootstraps
nog4_combined_bootstraps <- lapply(promoter_nog4_combined, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_bootstraps, file = 'RData/nog4_combined_bootstraps.RData')
lapply(nog4_combined_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load("RData/nog4_combined_bootstraps.RData")

# # non-G4 delta-G bootstraps
nog4_combined_delta_bootstraps <-  lapply(promoter_nog4_combined_delta, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_delta, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_delta_bootstraps, file = 'RData/nog4_combined_delta_bootstraps.RData')
lapply(nog4_combined_delta_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_delta_bootstraps.RData')

# # non-G4 const-G bootstraps
nog4_combined_const_bootstraps <-  lapply(promoter_nog4_combined_const, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_const, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_const_bootstraps, file = 'RData/nog4_combined_const_bootstraps.RData')
lapply(nog4_combined_const_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_const_bootstraps.RData')

# non-G4 stable-G bootstraps
nog4_combined_stable_bootstraps <-  lapply(promoter_nog4_combined_stable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))
save(nog4_combined_stable_bootstraps, file = 'RData/nog4_combined_stable_bootstraps.RData')
lapply(nog4_combined_stable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_stable_bootstraps.RData')

# non-G4 unstable-G bootstraps
nog4_combined_unstable_bootstraps <-  lapply(promoter_nog4_combined_unstable, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_delta, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_unstable_bootstraps, file = 'RData/nog4_combined_unstable_bootstraps.RData')
lapply(nog4_combined_unstable_bootstraps, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_unstable_bootstraps.RData')
       
# # g4 in PLOF genes bootstrap
combined_g4_plof_gruns_bootstrap <-  lapply(promoter_combined_g4_plof_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_plof_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_plof_gruns_bootstrap, file = 'RData/combined_g4_plof_gruns_bootstraps.RData')
lapply(combined_g4_plof_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load("RData/combined_g4_plof_gruns_bootstraps.RData")

# # nonG4 in PLOF genes bootstrap
combined_npg4_plof_gruns_bootstrap <-  lapply(nog4_combined_plof, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_plof, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_npg4_plof_gruns_bootstrap, file = 'RData/combined_npg4_plof_gruns_bootstrap.RData')
lapply(combined_npg4_plof_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_npg4_plof_gruns_bootstrap.RData')
# #
# G4 in disease gene bootstrap
combined_g4_clinvar_signal_bootstrap <-  lapply(combined_g4_clinvar_signal_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))
#lapply(combined_g4_signal_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_clinvar_signal_bootstrap, file = 'RData/combined_g4_clinvar_signal_bootstrap.RData')
lapply(combined_g4_clinvar_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_clinvar_signal_bootstrap.RData')

# # G4 in cgcs gene bootstrap
combined_g4_cgcs_signal_bootstrap <-  lapply(combined_g4_cgcs_signal_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_cgcs_signal_bootstrap, file = 'RData/combined_g4_cgcs_signal_bootstrap.RData')
lapply(combined_g4_cgcs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_cgcs_signal_bootstrap.RData')

# # G4 in disease gene bootstrap
combined_g4_tsgs_signal_bootstrap <-  lapply(combined_g4_tsgs_signal_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_gruns, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(combined_g4_tsgs_signal_bootstrap, file = 'RData/combined_g4_tsgs_signal_bootstrap.RData')
lapply(combined_g4_tsgs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_tsgs_signal_bootstrap.RData')

# # nonG4 in disease gene bootstrap
nog4_combined_clinvar_signal_bootstrap <-  lapply(nog4_combined_clinvar_signal, function(x) bootstrap_maps2(x, mu = mu, ps_lm= ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_signal, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_clinvar_signal_bootstrap, file = 'RData/nog4_combined_clivar_signal_bootstrap.RData')
lapply(nog4_combined_clinvar_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_clivar_signal_bootstrap.RData')
# #
# # nonG4 in disease gene bootstrap
nog4_combined_cgcs_signal_bootstrap <-  lapply(nog4_combined_cgcs_signal, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_signal, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_cgcs_signal_bootstrap, file = 'RData/nog4_combined_cgcs_signal_bootstrap.RData')
lapply(nog4_combined_cgcs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_cgcs_signal_bootstrap.RData')
# #
# # nonG4 in disease gene bootstrap
nog4_combined_tsgs_signal_bootstrap <-  lapply(nog4_combined_tsgs_signal, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(nog4_combined_signal, function(x) generate_bootstrap(x, 10000, mu = mu, ps_model = ps_lm))
save(nog4_combined_tsgs_signal_bootstrap, file = 'RData/nog4_combined_tsgs_signal_bootstrap.RData')
lapply(nog4_combined_tsgs_signal_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/nog4_combined_tsgs_signal_bootstrap.RData')

# # delta g4 in disease genes bootstrap
combined_g4_clinvar_signal_delta_gruns_bootstrap <-  lapply(combined_g4_clinvar_signal_delta_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_delta_gruns,
save(combined_g4_clinvar_signal_delta_gruns_bootstrap, file = 'RData/combined_g4_clinvar_signal_delta_gruns_bootstrap.RData')
lapply(combined_g4_clinvar_signal_delta_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_clinvar_signal_delta_gruns_bootstrap.RData')

# # delta g4 in cgcs genes bootstrap
combined_g4_cgcs_signal_delta_gruns_bootstrap <-  lapply(combined_g4_cgcs_signal_delta_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_delta_gruns,
save(combined_g4_cgcs_signal_delta_gruns_bootstrap, file = 'RData/combined_g4_cgcs_signal_delta_gruns_bootstrap.RData')
lapply(combined_g4_cgcs_signal_delta_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_cgcs_signal_delta_gruns_bootstrap.RData')

# # delta g4 in tsgs genes bootstrap
combined_g4_tsgs_signal_delta_gruns_bootstrap <-  lapply(combined_g4_tsgs_signal_delta_gruns, function(x) bootstrap_maps2(x, mu = mu, ps_lm = ps_lm_genomes, ntimes = 10000))#lapply(combined_g4_signal_delta_gruns,
save(combined_g4_tsgs_signal_delta_gruns_bootstrap, file = 'RData/combined_g4_tsgs_signal_delta_gruns_bootstrap.RData')
lapply(combined_g4_tsgs_signal_delta_gruns_bootstrap, function(x) quantile(x, c(0.05, 0.95)))
# load('RData/combined_g4_tsgs_signal_delta_gruns_bootstrap.RData')

# # ALL promoter variants bootstrap
promoter_gnomad_grange_bootstrap <- chunked_bootstrap(promoter_gnomad_grange, 10000, mu = mu, ps_model = ps_lm_genomes)
save(promoter_gnomad_grange_bootstrap, file = 'RData/promoter_gnomad_grange_bootstrap.RData')
quantile(promoter_gnomad_grange_bootstrap, c(0.05, 0.95))
# load('RData/promoter_gnomad_grange_bootstrap.RData')

# # ALL promoter variants delta pG4 bootstrap
combined_promoter_delta_variants_bootstrap <- chunked_bootstrap(all_promoter_delta_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_delta_variants_bootstrap, file = 'RData/combined_promoter_delta_variants_bootstrap.RData')
quantile(combined_promoter_delta_variants_bootstrap, c(0.05, 0.95))
# load('RData/combined_promoter_delta_variants_bootstrap.RData')

# # ALL promoter variants const pG4 bootstrap
combined_promoter_const_variants_bootstrap <- chunked_bootstrap(const_all_promoter_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_const_variants_bootstrap, file = 'RData/combined_promoter_const_variants_bootstrap.RData')
quantile(combined_promoter_const_variants_bootstrap, c(0.05, 0.95))
# load("RData/combined_promoter_const_variants_bootstrap.RData")

# ALL promoter variants stable pG4 bootstrap
combined_promoter_stable_variants_bootstrap <- chunked_bootstrap(all_promoter_stable_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_stable_variants_bootstrap, file = 'RData/combined_promoter_stable_variants_bootstrap.RData')
quantile(combined_promoter_stable_variants_bootstrap, c(0.05, 0.95))
# load('RData/combined_promoter_stable_variants_bootstrap.RData')

# ALL promoter variants unstable pG4 bootstrap
combined_promoter_unstable_variants_bootstrap <- chunked_bootstrap(all_promoter_unstable_variants, 10000, mu = mu, ps_model = ps_lm_genomes)
save(combined_promoter_unstable_variants_bootstrap, file = 'RData/combined_promoter_unstable_variants_bootstrap.RData')
quantile(combined_promoter_unstable_variants_bootstrap, c(0.05, 0.95))
# load('RData/combined_promoter_unstable_variants_bootstrap.RData')

# # All promoter variants plof bootstrap
all_promoter_plof_bootstrap <- chunked_bootstrap(all_promoter_plof, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_plof_bootstrap, file = 'RData/all_promoter_plof_bootstrap.RData')
quantile(all_promoter_plof_bootstrap, c(0.05, 0.95))
# load('RData/all_promoter_plof_bootstrap.RData')

# # ALL promoter variants disease genes bootstrap
all_promoter_clinvar_dx_bootstrap <- chunked_bootstrap(all_promoter_clinvar_signal, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_clinvar_dx_bootstrap, file = 'RData/all_promoter_clinvar_dx_bootstrap.RData')
quantile(all_promoter_clinvar_dx_bootstrap, c(0.05, 0.95))
# load('RData/all_promoter_clinvar_dx_bootstrap.RData')
#
# # ALL promoter variants disease genes bootstrap
all_promoter_cgcs_dx_bootstrap <- chunked_bootstrap(all_promoter_cgcs_signal, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_cgcs_dx_bootstrap, file = 'RData/all_promoter_cgcs_dx_bootstrap.RData')
quantile(all_promoter_cgcs_dx_bootstrap, c(0.05, 0.95))
# load('RData/all_promoter_cgcs_dx_bootstrap.RData')

# # ALL promoter variants disease genes bootstrap
all_promoter_tsgs_dx_bootstrap <- chunked_bootstrap(all_promoter_tsgs_signal, 10000, mu = mu, ps_model = ps_lm_genomes)
save(all_promoter_tsgs_dx_bootstrap, file = 'RData/all_promoter_tsgs_dx_bootstrap.RData')
quantile(all_promoter_tsgs_dx_bootstrap, c(0.05, 0.95))
# load('RData/all_promoter_tsgs_dx_bootstrap.RData')
                                      
confidence_intervals <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                             'pG4 G1' = g4_combined_bootstraps$G1,
                             'pG4 G2' = g4_combined_bootstraps$G2,
                             'pG4 G3' = g4_combined_bootstraps$G3,
                             'Top-third SNV' = all_promoter_plof_bootstrap,
                             'Top-third pG4 G1' = combined_g4_plof_gruns_bootstrap$G1,
                             'Top-third pG4 G2' = combined_g4_plof_gruns_bootstrap$G2,
                             'Top-third pG4 G3' = combined_g4_plof_gruns_bootstrap$G3,
                             'ClinVar SNV' = all_promoter_clinvar_dx_bootstrap,
                             'ClinVar pG4 G1' = combined_g4_clinvar_signal_bootstrap$G1,
                             'ClinVar pG4 G2' = combined_g4_clinvar_signal_bootstrap$G2,
                             'ClinVar pG4 G3' = combined_g4_clinvar_signal_bootstrap$G3,
                             'Cosmic SNV' = all_promoter_cgcs_dx_bootstrap,
                             'Cosmic pG4 G1' = combined_g4_cgcs_signal_bootstrap$G1,
                             'Cosmic pG4 G2' = combined_g4_cgcs_signal_bootstrap$G2,
                             'Cosmic pG4 G3' = combined_g4_cgcs_signal_bootstrap$G3,

                             'TsGenes SNV' = all_promoter_tsgs_dx_bootstrap,
                             'TsGenes pG4 G1' = combined_g4_tsgs_signal_bootstrap$G1,
                             'TsGenes pG4 G2' = combined_g4_tsgs_signal_bootstrap$G2,
                             'TsGenes pG4 G3' = combined_g4_tsgs_signal_bootstrap$G3
                             )

confidence_intervals <- sapply(confidence_intervals, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()

confidence_intervals$MAPS <- c(maps_all_promoter$all['MAPS'],
                               maps_pg4_combined$G1['MAPS'],
                               maps_pg4_combined$G2['MAPS'],
                               maps_pg4_combined$G3['MAPS'],
                               maps_plof_all_variants$all_promoter_plof['MAPS'],
                               maps_plof_gruns$G1['MAPS'],
                               maps_plof_gruns$G2['MAPS'],
                               maps_plof_gruns$G3['MAPS'],
                               maps_clinvar_signal_all_variants$clinvar['MAPS'],
                               maps_clinvar_pg4$G1['MAPS'],
                               maps_clinvar_pg4$G2['MAPS'],
                               maps_clinvar_pg4$G3['MAPS'],
                               maps_cgcs_signal_all_variants$cgcs["MAPS"],
                               maps_cgcs_pg4$G1['MAPS'],
                               maps_cgcs_pg4$G2["MAPS"],
                               maps_cgcs_pg4$G3["MAPS"],
                               maps_tsgs_signal_all_variants$tsgs["MAPS"],
                               maps_tsgs_pg4$G1["MAPS"],
                               maps_tsgs_pg4$G2["MAPS"],
                               maps_tsgs_pg4$G3["MAPS"]
                               )

plot_df <- confidence_intervals
plot_df$Category <- factor(c("All", "All", "All", "All",
                             "LOEUF Top 1/3", "LOEUF Top 1/3", "LOEUF Top 1/3", "LOEUF Top 1/3",
                             "ClinVar Genes", "ClinVar Genes", "ClinVar Genes", "ClinVar Genes",
                             "Cancer Gene Census","Cancer Gene Census","Cancer Gene Census","Cancer Gene Census",
                             "Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene"
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               length(all_promoter_plof),
               sapply(combined_g4_plof_gruns, function(x) length(x)),
               length(all_promoter_clinvar_signal),
               sapply(combined_g4_clinvar_signal_gruns, function(x) length(x)),
               length(all_promoter_cgcs_signal),
               sapply(combined_g4_cgcs_signal_gruns, function(x) length(x)),
               length(all_promoter_tsgs_signal),
               sapply(combined_g4_tsgs_signal_gruns, function(x) length(x))
)
plot_df$Context <- paste0(rep(c('All SNV','pG4 G1', 'pG4 G2', 'pG4 G3'), 5),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4'),5)

# tiff('figures/MAPS.tiff', units = 'in', width = 9, height = 6, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0)) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
#  geom_segment(x = 10, xend = 10, y = plot_df['ClinVar pG4 G1',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
#  geom_segment(x = 8, xend = 8, y = plot_df['Top-third pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
  geom_point() +
  ylim(c(-0.05, 0.10)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'red') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 1, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
        axis.title.y = element_text(),
        plot.margin=unit(c(0,0,0,0),"cm")) +
  annotate("text", x = 1.2, y = 0.043, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 1.4, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 1.0, y = 0.096, label = "pLOF", color = 'red', size = 2.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 3) +
  annotate("text", x = 6.5, y = -0.05, label = 'LOEUF Top 1/3', size = 3) +
  annotate("text", x = 10.5, y = -0.05, label = 'Disease Genes', size = 3)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 3)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor suppressor gene', size = 3)
ggsave('figures/MAPS.tiff', units = 'in', width = 9, height = 6, device=tiff())
ggsave('figures/MAPS.pdf', width = 9, height = 6,device=pdf())
dev.off()

###################################
## Plot delta pG4 vs. const. pG4 ##
###################################

confidence_intervals_2 <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                               'pG4 G1' = g4_combined_bootstraps$G1,
                               'pG4 G2' = g4_combined_bootstraps$G2,
                               'pG4 G3' = g4_combined_bootstraps$G3,
                               'Delta SNV' = combined_promoter_delta_variants_bootstrap,
                               'Delta pG4 G1' = g4_combined_delta_bootstraps$G1,
                               'Delta pG4 G2' = g4_combined_delta_bootstraps$G2,
                               'Delta pG4 G3' = g4_combined_delta_bootstraps$G3,
                               'ClinVar SNV' = all_promoter_clinvar_dx_bootstrap,
                               'ClinVar pG4 G1' = combined_g4_clinvar_signal_bootstrap$G1,
                               'ClinVar pG4 G2' = combined_g4_clinvar_signal_bootstrap$G2,
                               'ClinVar pG4 G3' = combined_g4_clinvar_signal_bootstrap$G3,
                               'Cosmic SNV' = all_promoter_cgcs_dx_bootstrap,
                               'Cosmic pG4 G1' = combined_g4_cgcs_signal_bootstrap$G1,
                               'Cosmic pG4 G2' = combined_g4_cgcs_signal_bootstrap$G2,
                               'Cosmic pG4 G3' = combined_g4_cgcs_signal_bootstrap$G3,
                               'TsGenes SNV' = all_promoter_tsgs_dx_bootstrap,
                               'TsGenes pG4 G1' = combined_g4_tsgs_signal_bootstrap$G1,
                               'TsGenes pG4 G2' = combined_g4_tsgs_signal_bootstrap$G2,
                               'TsGenes pG4 G3' = combined_g4_tsgs_signal_bootstrap$G3
                             )

confidence_intervals_2 <- sapply(confidence_intervals_2, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()
confidence_intervals_2$MAPS <- c(maps_all_promoter$all['MAPS'],
                                 maps_pg4_combined$G1['MAPS'],
                                 maps_pg4_combined$G2['MAPS'],
                                 maps_pg4_combined$G3['MAPS'],
                                 maps_all_promoter_delta$all['MAPS'],
                                 maps_promoter_combined_delta$G1['MAPS'],
                                 maps_promoter_combined_delta$G2['MAPS'],
                                 maps_promoter_combined_delta$G3['MAPS'],
                                 maps_clinvar_signal_all_variants$clinvar['MAPS'],
                                 maps_clinvar_pg4$G1['MAPS'],
                                 maps_clinvar_pg4$G2['MAPS'],
                                 maps_clinvar_pg4$G3['MAPS'],
                                 maps_cgcs_signal_all_variants$cgcs["MAPS"],
                                 maps_cgcs_pg4$G1['MAPS'],
                                 maps_cgcs_pg4$G2["MAPS"],
                                 maps_cgcs_pg4$G3["MAPS"],
                                 maps_tsgs_signal_all_variants$tsgs["MAPS"],
                                 maps_tsgs_pg4$G1["MAPS"],
                                 maps_tsgs_pg4$G2["MAPS"],
                                 maps_tsgs_pg4$G3["MAPS"]
                               )

plot_df <- confidence_intervals_2
plot_df$Category <- factor(c("All", "All", "All", "All",
                             "Alternative", "Alternative", "Alternative", "Alternative",
                             "ClinVar", "ClinVar", "ClinVar", "ClinVar",
                             "Cancer Gene Census","Cancer Gene Census","Cancer Gene Census","Cancer Gene Census",
                             "Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene","Tumor suppressor gene"
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               length(all_promoter_delta_variants),
               sapply(promoter_combined_delta, function(x) length(x)),
               length(all_promoter_clinvar_signal),
               sapply(combined_g4_clinvar_signal_gruns, function(x) length(x)),
               length(all_promoter_cgcs_signal),
               sapply(combined_g4_cgcs_signal_gruns, function(x) length(x)),
               length(all_promoter_tsgs_signal),
               sapply(combined_g4_tsgs_signal_gruns, function(x) length(x))
)

plot_df$Context <- paste0(rep(c('All SNV','pG4 G1', 'pG4 G2', 'pG4 G3'), 5),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4'),5)

# tiff('figures/MAPS_delta_dx.tiff', units = 'in', width = 9, height = 6, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0)) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.4) +
  geom_point() +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 1, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
        axis.title.y = element_text(),
        plot.margin=unit(c(0,0,0,0),"cm")) +
  annotate("text", x = 1, y = 0.043, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 1.2, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 0.8, y = 0.096, label = "pLOF", color = 'purple', size = 2.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 3) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 3) +
  annotate("text", x = 10.5, y = -0.05, label = 'Diseae Genes', size = 3)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 3)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor suppressor gene', size = 3)

ggsave('figures/MAPS_delta_dx.tiff', units = 'in', width = 9, height = 6,device=tiff())
ggsave('figures/MAPS_delta_dx.pdf', width = 9, height = 6, device=pdf())
dev.off()

tiff('figures/MAPS_delta_dx_resized.tiff', units = 'in', width = 6, height = 4, res = 600)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0), size = 0.25) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.2) +
  geom_point(size = 0.5) +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 0.5, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 4),
        axis.text.y = element_text(hjust = 1, size = 7),
        axis.title.y = element_text(size=5),
        plot.margin=unit(c(0,0,0,0),"cm"),
        legend.title = element_text(size=5),
        legend.text = element_text(size=5)) +
  annotate("text", x = 1.3, y = 0.043, label = "missense", color = 'orange', size = 1.5) +
  annotate("text", x = 1.55, y = 0.005, label = "synonymous", color = 'grey', size = 1.5) +
  annotate("text", x = 1, y = 0.096, label = "pLOF", color = 'purple', size = 1.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 2) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 2) +
  annotate("text", x = 10.5, y = -0.05, label = 'Disease Genes', size = 2)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 2)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor suppressor gene', size = 2)
dev.off()

# pdf('figures/MAPS_delta_dx_resized.pdf', width = 6, height = 4, res = 600)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0), size = 0.25) +
  # geom_segment(x = 12, xend = 12, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.2) +
  geom_point(size = 0.5) +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 0.5, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 5),
        axis.text.y = element_text(hjust = 1, size = 9),
        axis.title.y = element_text(size=9),
        plot.margin=unit(c(0,0,0,0),"cm"),
        legend.title = element_text(size=5),
        legend.text = element_text(size=5)) +
  annotate("text", x = 1.3, y = 0.043, label = "missense", color = 'orange', size = 1.5) +
  annotate("text", x = 1.55, y = 0.005, label = "synonymous", color = 'grey', size = 1.5) +
  annotate("text", x = 1, y = 0.096, label = "pLOF", color = 'purple', size = 1.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All Promoters', size = 2) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 2) +
  annotate("text", x = 10.5, y = -0.05, label = 'Disease Genes', size = 2)+
  annotate("text", x = 14.5, y = -0.05, label = 'Cancer Gene Census', size = 2)+
  annotate("text", x = 18.5, y = -0.05, label = 'Tumor Suppressor Gene', size = 2)
# grDevices::embedFonts('figures/MAPS_delta_dx_resized.pdf')
ggsave('figures/MAPS_delta_dx_resized.pdf', width = 6, height = 4,device = pdf())
ggsave('figures/MAPS_delta_dx_resized.tiff', width = 6, height = 4,device = tiff())
dev.off()

###########################################
## Plot all G4, delta G4, and clinvar G4 ##
###########################################

confidence_intervals_2 <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                               'pG4 G1' = g4_combined_bootstraps$G1,
                               'pG4 G2' = g4_combined_bootstraps$G2,
                               'pG4 G3' = g4_combined_bootstraps$G3,
                               'Delta SNV' = combined_promoter_delta_variants_bootstrap,
                               'Delta pG4 G1' = g4_combined_delta_bootstraps$G1,
                               'Delta pG4 G2' = g4_combined_delta_bootstraps$G2,
                               'Delta pG4 G3' = g4_combined_delta_bootstraps$G3,
                               'Const. SNV' = combined_promoter_const_variants_bootstrap,
                               'Const. pG4 G1' = g4_combined_const_bootstraps$G1,
                               'Const. pG4 G2' = g4_combined_const_bootstraps$G2,
                               'Const. pG4 G3' = g4_combined_const_bootstraps$G3,

                               'Stable SNV' = combined_promoter_stable_variants_bootstrap,
                               'Stable pG4 G1' = g4_combined_stable_bootstraps$G1,
                               'Stable pG4 G2' = g4_combined_stable_bootstraps$G2,
                               'Stable pG4 G3' = g4_combined_stable_bootstraps$G3,

                               'Unstable SNV' = combined_promoter_unstable_variants_bootstrap,
                               'Unstable pG4 G1' = g4_combined_unstable_bootstraps$G1,
                               'Unstable pG4 G2' = g4_combined_unstable_bootstraps$G2,
                               'Unstable pG4 G3' = g4_combined_unstable_bootstraps$G3

)

confidence_intervals_2 <- sapply(confidence_intervals_2, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()

confidence_intervals_2$MAPS <- c(maps_all_promoter$all['MAPS'],
                                 maps_pg4_combined$G1['MAPS'],
                                 maps_pg4_combined$G2['MAPS'],
                                 maps_pg4_combined$G3['MAPS'],

                                 maps_all_promoter_delta$all['MAPS'],
                                 maps_promoter_combined_delta$G1['MAPS'],
                                 maps_promoter_combined_delta$G2['MAPS'],
                                 maps_promoter_combined_delta$G3['MAPS'],

                                 maps_all_promoter_const$all['MAPS'],
                                 maps_promoter_combined_const$G1['MAPS'],
                                 maps_promoter_combined_const$G2['MAPS'],
                                 maps_promoter_combined_const$G3['MAPS'],

                                 maps_all_promoter_stable$all['MAPS'],
                                 maps_promoter_combined_stable$G1['MAPS'],
                                 maps_promoter_combined_stable$G2['MAPS'],
                                 maps_promoter_combined_stable$G3['MAPS'],

                                 maps_all_promoter_unstable$all['MAPS'],
                                 maps_promoter_combined_unstable$G1['MAPS'],
                                 maps_promoter_combined_unstable$G2['MAPS'],
                                 maps_promoter_combined_unstable$G3['MAPS']

                                 )

plot_df <- confidence_intervals_2
plot_df$Category <- factor(c("All", "All", "All", "All",
                             "Alternative", "Alternative", "Alternative", "Alternative",
                             "Const.", "Const.", "Const.", "Const.",
                             "Stable.", "Stable.", "Stable.", "Stable.",
                             "Unstable.", "Unstable.", "Unstable.", "Unstable."
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               length(all_promoter_delta_variants),
               sapply(promoter_combined_delta, function(x) length(x)),
               length(const_all_promoter_variants),
               sapply(promoter_combined_const, function(x) length(x)),
               length(all_promoter_stable_variants),
               sapply(promoter_combined_stable, function(x) length(x)),

               length(all_promoter_unstable_variants),
               sapply(promoter_combined_unstable, function(x) length(x))
               )

plot_df$Context <- paste0(rep(c('All SNV','G4 G1', 'G4 G2', 'G4 G3'), 5),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4'),5)
# tiff('figures/MAPS_delta_const.tiff', units = 'in', width = 7, height = 5, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0)) +
  geom_point() +
  ylim(c(-0.05, .12)) + ylab(expression(atop("Increasing Negative Selection  " ~ symbol('\256'),"MAPS"))) +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red")) +
  theme(axis.line = element_line(size = 1, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 45, hjust = 1, size = 9),
        axis.title.y = element_text(),
        plot.margin=unit(c(0,0,0,0),"cm")) +
  annotate("text", x = 1, y = 0.044, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 1.2, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 0.8, y = 0.096, label = "pLOF", color = 'purple', size = 2.5) +
  geom_vline(xintercept = 4.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 8.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 12.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 16.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 2.5, y = -0.05, label = 'All UTRs', size = 3) +
  annotate("text", x = 6.5, y = -0.05, label = 'Alt. pG4', size = 3) +
  annotate("text", x = 10.5, y = -0.05, label = 'Cons. pG4', size = 3) +
  annotate("text", x = 14.5, y = -0.05, label = 'Stable. UTRs', size = 3) +
  annotate("text", x = 18.5, y = -0.05, label = 'Unstable. pG4', size = 3)
ggsave('figures/MAPS_delta_const.tiff', units = 'in', width = 12, height = 5, dpi = 300,device=tiff())
ggsave('figures/MAPS_delta_const.pdf', width = 12, height = 5, device=pdf())
dev.off()


###########################################
## Plot with comparison of non-pG4 Gruns ##
###########################################

confidence_intervals_2 <- list('All SNV' = promoter_gnomad_grange_bootstrap,
                               'pG4 G1' = g4_combined_bootstraps$G1,
                               'pG4 G2' = g4_combined_bootstraps$G2,
                               'pG4 G3' = g4_combined_bootstraps$G3,
                               'non-G4 1' = nog4_combined_bootstraps$pos1,
                               'non-G4 2' = nog4_combined_bootstraps$pos2,
                               'non-G4 3' = nog4_combined_bootstraps$pos3,
                               'plof All SNV' = all_promoter_plof_bootstrap,
                               'plof pG4 G1' = combined_g4_plof_gruns_bootstrap$G1,
                               'plof pG4 G2' = combined_g4_plof_gruns_bootstrap$G2,
                               'plof pG4 G3' = combined_g4_plof_gruns_bootstrap$G3,
                               'plof non-G4 1' = combined_npg4_plof_gruns_bootstrap$pos1,
                               'plof non-G4 2' = combined_npg4_plof_gruns_bootstrap$pos2,
                               'plof non-G4 3' = combined_npg4_plof_gruns_bootstrap$pos3,
                               'Delta SNV' = combined_promoter_delta_variants_bootstrap,
                               'Delta pG4 G1' = g4_combined_delta_bootstraps$G1,
                               'Delta pG4 G2' = g4_combined_delta_bootstraps$G2,
                               'Delta pG4 G3' = g4_combined_delta_bootstraps$G3,
                               'Delta non-G4 1' = nog4_combined_delta_bootstraps$pos1,
                               'Delta non-G4 2' = nog4_combined_delta_bootstraps$pos2,
                               'Delta non-G4 3' = nog4_combined_delta_bootstraps$pos3,
                               'Const. SNV' = combined_promoter_const_variants_bootstrap,
                               'Const. pG4 G1' = g4_combined_const_bootstraps$G1,
                               'Const. pG4 G2' = g4_combined_const_bootstraps$G2,
                               'Const. pG4 G3' = g4_combined_const_bootstraps$G3,
                               'Const. non-G4 1' = nog4_combined_const_bootstraps$pos1,
                               'Const. non-G4 2' = nog4_combined_const_bootstraps$pos2,
                               'Const. non-G4 3' = nog4_combined_const_bootstraps$pos3,

                               'Stable SNV' = combined_promoter_stable_variants_bootstrap,
                               'Stable pG4 G1' = g4_combined_stable_bootstraps$G1,
                               'Stable pG4 G2' = g4_combined_stable_bootstraps$G2,
                               'Stable pG4 G3' = g4_combined_stable_bootstraps$G3,
                               'Stable non-G4 1' = nog4_combined_stable_bootstraps$pos1,
                               'Stable non-G4 2' = nog4_combined_stable_bootstraps$pos2,
                               'Stable non-G4 3' = nog4_combined_stable_bootstraps$pos3,


                               'Unstable SNV' = combined_promoter_unstable_variants_bootstrap,
                               'Unstable pG4 G1' = g4_combined_unstable_bootstraps$G1,
                               'Unstable pG4 G2' = g4_combined_unstable_bootstraps$G2,
                               'Unstable pG4 G3' = g4_combined_unstable_bootstraps$G3,
                               'Unstable non-G4 1' = nog4_combined_unstable_bootstraps$pos1,
                               'Unstable non-G4 2' = nog4_combined_unstable_bootstraps$pos2,
                               'Unstable non-G4 3' = nog4_combined_unstable_bootstraps$pos3,

                               'ClinVar SNV' = all_promoter_clinvar_dx_bootstrap,
                               'ClinVar pG4 G1' = combined_g4_clinvar_signal_bootstrap$G1,
                               'ClinVar pG4 G2' = combined_g4_clinvar_signal_bootstrap$G2,
                               'ClinVar pG4 G3' = combined_g4_clinvar_signal_bootstrap$G3,
                               'ClinVar non-G4 1' = nog4_combined_clinvar_signal_bootstrap$pos1,
                               'ClinVar non-G4 2' = nog4_combined_clinvar_signal_bootstrap$pos2,
                               'ClinVar non-G4 3' = nog4_combined_clinvar_signal_bootstrap$pos3,

                               'Cosmic SNV' = all_promoter_cgcs_dx_bootstrap,
                               'Cosmic pG4 G1' = combined_g4_cgcs_signal_bootstrap$G1,
                               'Cosmic pG4 G2' = combined_g4_cgcs_signal_bootstrap$G2,
                               'Cosmic pG4 G3' = combined_g4_cgcs_signal_bootstrap$G3,
                               'Cosmic non-G4 1' = nog4_combined_cgcs_signal_bootstrap$pos1,
                               'Cosmic non-G4 2' = nog4_combined_cgcs_signal_bootstrap$pos2,
                               'Cosmic non-G4 3' = nog4_combined_cgcs_signal_bootstrap$pos3,

                               'TsGenes SNV' = all_promoter_tsgs_dx_bootstrap,
                               'TsGenes pG4 G1' = combined_g4_tsgs_signal_bootstrap$G1,
                               'TsGenes pG4 G2' = combined_g4_tsgs_signal_bootstrap$G2,
                               'TsGenes pG4 G3' = combined_g4_tsgs_signal_bootstrap$G3,
                               'TsGenes non-G4 1' = nog4_combined_tsgs_signal_bootstrap$pos1,
                               'TsGenes non-G4 2' = nog4_combined_tsgs_signal_bootstrap$pos2,
                               'TsGenes non-G4 3' = nog4_combined_tsgs_signal_bootstrap$pos3
)
confidence_intervals_2 <- sapply(confidence_intervals_2, function(x) quantile(x, c(0.05, 0.95))) %>%
  t() %>%
  as.data.frame()

confidence_intervals_2$MAPS <- c(maps_all_promoter$all['MAPS'],
                                 maps_pg4_combined$G1['MAPS'],
                                 maps_pg4_combined$G2['MAPS'],
                                 maps_pg4_combined$G3['MAPS'],
                                 maps_nong4_first_last$pos1['MAPS'],
                                 maps_nong4_first_last$pos2['MAPS'],
                                 maps_nong4_first_last$pos3['MAPS'],

                                 maps_plof_all_variants$all_promoter_plof['MAPS'],
                                 maps_plof_gruns$G1['MAPS'],
                                 maps_plof_gruns$G2['MAPS'],
                                 maps_plof_gruns$G3['MAPS'],
                                 maps_nog4_combined_plof$pos1['MAPS'],
                                 maps_nog4_combined_plof$pos2['MAPS'],
                                 maps_nog4_combined_plof$pos3['MAPS'],

                                 maps_all_promoter_delta$all['MAPS'],
                                 maps_promoter_combined_delta$G1['MAPS'],
                                 maps_promoter_combined_delta$G2['MAPS'],
                                 maps_promoter_combined_delta$G3['MAPS'],
                                 maps_promoter_nog4_delta$pos1['MAPS'],
                                 maps_promoter_nog4_delta$pos2['MAPS'],
                                 maps_promoter_nog4_delta$pos3['MAPS'],

                                 maps_all_promoter_const$all['MAPS'],
                                 maps_promoter_combined_const$G1['MAPS'],
                                 maps_promoter_combined_const$G2['MAPS'],
                                 maps_promoter_combined_const$G3['MAPS'],
                                 maps_promoter_nog4_const$pos1['MAPS'],
                                 maps_promoter_nog4_const$pos2['MAPS'],
                                 maps_promoter_nog4_const$pos3['MAPS'],

                                 maps_all_promoter_stable$all['MAPS'],
                                 maps_promoter_combined_stable$G1['MAPS'],
                                 maps_promoter_combined_stable$G2['MAPS'],
                                 maps_promoter_combined_stable$G3['MAPS'],
                                 maps_promoter_nog4_stable$pos1['MAPS'],
                                 maps_promoter_nog4_stable$pos2['MAPS'],
                                 maps_promoter_nog4_stable$pos3['MAPS'],

                                 maps_all_promoter_unstable$all['MAPS'],
                                 maps_promoter_combined_unstable$G1['MAPS'],
                                 maps_promoter_combined_unstable$G2['MAPS'],
                                 maps_promoter_combined_unstable$G3['MAPS'],
                                 maps_promoter_nog4_unstable$pos1['MAPS'],
                                 maps_promoter_nog4_unstable$pos2['MAPS'],
                                 maps_promoter_nog4_unstable$pos3['MAPS'],

                                 maps_clinvar_signal_all_variants$clinvar['MAPS'],
                                 maps_clinvar_pg4$G1['MAPS'],
                                 maps_clinvar_pg4$G2['MAPS'],
                                 maps_clinvar_pg4$G3['MAPS'],
                                 maps_clinvar_nong4$pos1['MAPS'],
                                 maps_clinvar_nong4$pos2['MAPS'],
                                 maps_clinvar_nong4$pos3['MAPS'],

                                 maps_cgcs_signal_all_variants$cgcs['MAPS'],
                                 maps_cgcs_pg4$G1['MAPS'],
                                 maps_cgcs_pg4$G2['MAPS'],
                                 maps_cgcs_pg4$G3['MAPS'],
                                 maps_cgcs_nong4$pos1['MAPS'],
                                 maps_cgcs_nong4$pos2['MAPS'],
                                 maps_cgcs_nong4$pos3['MAPS'],

                                 maps_tsgs_signal_all_variants$tsgs['MAPS'],
                                 maps_tsgs_pg4$G1['MAPS'],
                                 maps_tsgs_pg4$G2['MAPS'],
                                 maps_tsgs_pg4$G3['MAPS'],
                                 maps_tsgs_nong4$pos1['MAPS'],
                                 maps_tsgs_nong4$pos2['MAPS'],
                                 maps_tsgs_nong4$pos3['MAPS']
)
plot_df <- confidence_intervals_2
plot_df$Category <- factor(c(rep("All", 7),
                             rep('Top 1/3 LOEUF', 7),
                             rep("Alternative", 7),
                             rep('Constitutive', 7),
                             rep("Stable", 7),
                             rep('Unstable', 7),
                             rep('ClinVar', 7),
                             rep('Cancer Gene Cencus',7),
                             rep('Tumor Suppressor Genes',7)
                             ),
                           ordered = TRUE)
plot_df$N <- c(length(promoter_gnomad_grange),
               sapply(promoter_combined, function(x) length(x)),
               sapply(promoter_nog4_combined, function(x) length(x)),

               length(all_promoter_plof),
               sapply(combined_g4_plof_gruns, function(x) length(x)),
               sapply(nog4_combined_plof, function(x) length(x)),

               length(all_promoter_delta_variants),
               sapply(promoter_combined_delta, function(x) length(x)),
               sapply(promoter_nog4_combined_delta, function(x) length(x)),

               length(const_all_promoter_variants),
               sapply(promoter_combined_const, function(x) length(x)),
               sapply(promoter_nog4_combined_const, function(x) length(x)),

               length(all_promoter_stable_variants),
               sapply(promoter_combined_stable, function(x) length(x)),
               sapply(promoter_nog4_combined_stable, function(x) length(x)),

               length(all_promoter_unstable_variants),
               sapply(promoter_combined_unstable, function(x) length(x)),
               sapply(promoter_nog4_combined_unstable, function(x) length(x)),

               length(all_promoter_clinvar_signal),
               sapply(combined_g4_clinvar_signal_delta_gruns, function(x) length(x)),
               sapply(nog4_combined_clinvar_signal, function(x) length(x)),

               length(all_promoter_cgcs_signal),
               sapply(combined_g4_cgcs_signal_delta_gruns, function(x) length(x)),
               sapply(nog4_combined_cgcs_signal, function(x) length(x)),

               length(all_promoter_tsgs_signal),
               sapply(combined_g4_tsgs_signal_delta_gruns, function(x) length(x)),
               sapply(nog4_combined_tsgs_signal, function(x) length(x))
               )
plot_df$Context <- paste0(rep(c('All SNV','pG4 G1', 'pG4 G2', 'pG4 G3', 'non-G4 1', 'non-G4 2', 'non-G4 3'), 9),
                          '\n(N = ',
                          prettyNum(plot_df$N, big.mark=',', trim = TRUE), ')')

plot_df$Context <- factor(plot_df$Context, levels = plot_df$Context)

plot_df$Class <- rep(c('All SNVs', 'G4', 'G4', 'G4', 'non-G4', 'non-G4', 'non-G4'),9)

# tiff('figures/MAPS_all_classes_3.tiff', units = 'in', width = 10, height = 4, res = 300)
ggplot(plot_df, aes(x = Context, y = MAPS, color = Class)) +
  geom_errorbar(aes(ymin=`5%`, ymax=`95%`, width=0), size = 0.25) +
  # geom_segment(x = 32, xend = 32, y = plot_df['ClinVar pG4 G3',]$`95%`, yend = -Inf, color = 'red', size = 0.2) +
  geom_point(size = 0.5) +
  ylim(c(-0.05, .12)) + ylab("MAPS") +
  xlab('') +
  geom_hline(yintercept = 0, lty = 2, color = 'grey') +
  geom_hline(yintercept = maps_cds_markers$missense_all['MAPS'], lty = 2, color = 'orange') +
  geom_hline(yintercept = maps_cds_markers$lof_all['MAPS'], lty = 2, color = 'purple') +
  scale_color_manual(values = c("black", "red", "blue")) +
  theme(axis.line = element_line(size = 0.5, color = 'black'),
        panel.background = element_rect((fill = 'white')),
        axis.text.x = element_text(angle = 65, hjust = 1, size = 5),
        axis.title.y = element_text(size = 8),
        axis.text.y = element_text(size = 8),
        plot.margin=unit(c(0,0,0,0),"cm"),
        legend.position = 'top',
        legend.title = element_blank(),
        legend.text = element_text(size=5)) +
  annotate("text", x = 2, y = 0.043, label = "missense", color = 'orange', size = 2.5) +
  annotate("text", x = 2, y = 0.005, label = "synonymous", color = 'grey', size = 2.5) +
  annotate("text", x = 1.5, y = 0.096, label = "pLOF", color = 'purple', size = 2.5) +
  geom_vline(xintercept = 7.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 14.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 15 + 6.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 28.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 35.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 42.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 49.5, lty = 1, color = 'darkgrey', size = 0.3) +
  geom_vline(xintercept = 56.5, lty = 1, color = 'darkgrey', size = 0.3) +
  annotate("text", x = 3.75, y = -0.05, label = 'All Promiters', size = 2) +
  annotate("text", x = 11.25, y = -0.05, label = 'Top 1/3 LOEUF', size = 2) +
  annotate("text", x = 17.75, y = -0.05, label = 'Alt. pG4', size = 2) +
  annotate("text", x = 25.25, y = -0.05, label = 'Const. pG4', size = 2) +
  annotate("text", x = 24.25 + 7.5, y = -0.05, label = 'Stable pG4', size = 2) +
  annotate("text", x = 32.25 + 7, y = -0.05, label = 'Unstable pG4', size = 2) +
  annotate("text", x = 39.25 + 7.5, y = -0.05, label = 'ClinVar pG4', size = 2)+
  annotate("text", x = 46.25 + 7, y = -0.05, label = 'Cancer Gene Cencus pG4', size = 2)+
  annotate("text", x = 54.25 + 7, y = -0.05, label = 'Tumor Suppressor Genes pG4', size = 2)
ggsave('figures/MAPS_all_classes_3.tiff', units = 'in', width = 10, height = 4, dpi = 300,device = tiff())
ggsave('figures/MAPS_all_classes_3.pdf', width = 10, height = 4, device = pdf())
dev.off()


## R PopGenome analysis

In [ ]:
# -*- coding: utf-8 -*-
# @Time    : 2022/11/8 9:30
# @Author  : Guangyuel


import pandas as pd
import numpy as np
import os
import sys
import pyranges as pr


def file_check(path):
    if not os.path.isfile(path):
        sys,exit(f"this file {path} is not exist !!!")
    else:
        return path


def promoter_select(promoter, promoter_pG4, promoter_name, gruns, WD_PATH):
    promoter_pG4 = promoter_pG4.overlap(promoter, strandedness=False)
    promoter_pG4_Stable = promoter_pG4[promoter_pG4.Score > 19]
    promoter_pG4_unStable = promoter_pG4[promoter_pG4.Score <= 19]
    promoter_noG4 = promoter.subtract(promoter_pG4, strandedness=False).merge()
    promoter_pG4.to_csv(os.path.join(WD_PATH, promoter_name + "_pG4.info.tsv"), sep="\t")
    promoter_pG4_Stable.to_csv(os.path.join(WD_PATH, promoter_name + "_pG4_Stable.info.tsv"), sep="\t")
    promoter_pG4_unStable.to_csv(os.path.join(WD_PATH, promoter_name + "_pG4_unStable.info.tsv"), sep="\t")
    promoter_noG4.to_csv(os.path.join(WD_PATH, promoter_name + "_noG4.info.tsv"), sep="\t")
    # G-tracts analysis
    promoter_G4_Gtract = gruns.overlap(promoter_pG4, strandedness="same")
    promoter_stableG4_Gtract = gruns.overlap(promoter_pG4_Stable, strandedness="same")
    promoter_unstable_Gtracts = gruns.overlap(promoter_pG4_unStable, strandedness="same")
    promoter_nonG4_Gtract = gruns.overlap(promoter_noG4, strandedness=False)
    promoter_G4_Gtract.to_bed(os.path.join(WD_PATH,promoter_name + "_pG4_Gtract.bed"))
    promoter_stableG4_Gtract.to_bed(os.path.join(WD_PATH, promoter_name + "_stableG4_Gtract.bed"))
    promoter_unstable_Gtracts.to_bed(os.path.join(WD_PATH, promoter_name + "_unstable_Gtracts.bed"))
    promoter_nonG4_Gtract.to_bed(os.path.join(WD_PATH,promoter_name + "_nonG4_Gtract.bed"))


WD_PATH = "/disk2/G4_MS/frequency/envolution/data"
DATA_PTAH = "/disk2/G4_MS/promoter_select/promoter_info"
REFERENCE_PTAH = "/disk2/G4_MS/reference"

promoter_pG4_path = file_check(os.path.join(DATA_PTAH, "promoter_pG4.info.tsv"))
promoter_pG4 = pr.PyRanges(pd.read_table(promoter_pG4_path))
promoter_path = file_check(os.path.join(DATA_PTAH, "promoter.annotion.tsv"))
promoter = pr.PyRanges(pd.read_table(promoter_path))

cds_path = file_check(os.path.join(REFERENCE_PTAH, "gencode38.cds_region.bed"))
cds = pr.read_bed(cds_path)

gruns_path = file_check(os.path.join(REFERENCE_PTAH, "genomic_Gruns.bed"))
gruns = pr.PyRanges(pd.read_table(gruns_path, header=None, names=["Chromosome", "Start", "End", "Strand", "Gtract"]))
# promoter_pG4 = promoter_pG4.overlap(cds, strandedness=False, invert=True)
promoter = promoter.subtract(cds, strandedness=False).max_disjoint(strand=False)
promoter.to_bed(os.path.join(WD_PATH, "promoter.bed"))
promoter_select(promoter, promoter_pG4, "promoter", gruns, WD_PATH)
## activate promoter
active_promoter_path = file_check(os.path.join(DATA_PTAH, "promoter_info.annotion.H3K4me3.tsv"))
active_promoter = pr.PyRanges(pd.read_table(active_promoter_path))
active_promoter_pG4_path = file_check(os.path.join(DATA_PTAH, "promoter_pG4.info.H3K4me3.tsv"))
active_promoter_pG4 = pr.PyRanges(pd.read_table(active_promoter_pG4_path))
active_promoter = active_promoter.subtract(cds, strandedness=False).max_disjoint(strand=False)
active_promoter.to_bed(os.path.join(WD_PATH, "active_promoter.bed"))
promoter_select(active_promoter, active_promoter_pG4, "active_promoter", gruns, WD_PATH)

In [ ]:
%%bash
cd /disk2/G4_MS/frequency/envolution
cat promoter.bed active_promoter.bed |cut -f1-3 |bedtools sort -i -|bedtools merge -i - > promoter.merge.bed
awk -v OFS="\t" '{print$1,$2,$3-1}' promoter.merge.bed > promoter.merge.bed1
mv promoter.merge.bed1 promoter.merge_bed
for i in chr{{1..22},X,Y};do 
bcftools filter -R promoter.merge.bed -O z ../CCDG_13607_B01_GRM_WGS_2019-02-19_${i}.recalibrated_variants.norm_PASS_SNP.gz -o CCDG_13607_B01_GRM_WGS_2019-02-19_${i}.recalibrated_variants.norm_PASS_SNP.gz;done
# bcftools concat  ../*recalibrated_variants.norm_PASS_SNP.gz -O z -o CCDG_13607_B01_GRM_WGS_2019-02-19_merge.recalibrated_variants.norm_PASS_SNP.gz
bcftools concat  *recalibrated_variants.norm_PASS_SNP.gz -O z -R promoter.merge.bed -a -o CCDG_13607_B01_GRM_WGS_2019-02-19_merge.recalibrated_variants.norm_PASS_SNP.gz
cut -f 1-3 promoter.bed |bedtools sort -i - |bedtools merge -i - | awk -v OFS="\t" '{print$1,$2,$3-1}'> promoter_bed
cut -f 1-3 active_promoter.bed |bedtools sort -i - |bedtools merge -i - | awk -v OFS="\t" '{print$1,$2,$3-1}'> active_promoter_bed
bcftools filter -R promoter.bed -O z CCDG_13607_B01_GRM_WGS_2019-02-19_merge.recalibrated_variants.norm_PASS_SNP.gz -o promoter.kg.norm_PASS_SNP.vcf.gz
bcftools filter -R active_promoter.bed -O z CCDG_13607_B01_GRM_WGS_2019-02-19_merge.recalibrated_variants.norm_PASS_SNP.gz -o active_promoter.kg.norm_PASS_SNP.vcf.gz
bcftools index -t active_promoter.kg.norm_PASS_SNP.vcf.gz 
bcftools index -t promoter.kg.norm_PASS_SNP.vcf.gz

In [ ]:
%%bash
cd /disk2/G4_MS/frequency/evolution/analysis
tail -n +2 ../data/promoter_pG4.info.tsv |cut -f 1-3|bedtools sort -i -|bedtools merge -i -|awk -v OFS="\t" '{print $1,$2-1,$3-1}' > promoter_pG4.bed
tail -n +2 ../data/promoter_pG4_Stable.info.tsv |cut -f 1-3|bedtools sort -i -|bedtools merge -i -|awk -v OFS="\t" '{print $1,$2-1,$3-1}' > promoter_pG4_Stable.bed
tail -n +2 ../data/promoter_pG4_unStable.info.tsv |cut -f 1-3|bedtools sort -i -|bedtools merge -i -|awk -v OFS="\t" '{print $1,$2-1,$3-1}' > promoter_pG4_unStable.bed
tail -n +2 ../data/active_promoter_pG4.info.tsv |cut -f 1-3|bedtools sort -i -|bedtools merge -i -|awk -v OFS="\t" '{print $1,$2-1,$3-1}' > active_promoter_pG4.bed
tail -n +2 ../data/active_promoter_pG4_Stable.info.tsv |cut -f 1-3|bedtools sort -i -|bedtools merge -i -|awk -v OFS="\t" '{print $1,$2-1,$3-1}' > active_promoter_pG4_Stable.bed
tail -n +2 ../data/active_promoter_pG4_unStable.info.tsv |cut -f 1-3|bedtools sort -i -|bedtools merge -i -|awk -v OFS="\t" '{print $1,$2-1,$3-1}' > active_promoter_pG4_unStable.bed

awk -v OFS="\t" '{print $1,$2-1,$3-1}' ../data/promoter_pG4_Gtract.bed > promoter_pG4_Gtract.bed
awk -v OFS="\t" '{print $1,$2-1,$3-1}' ../data/promoter_stableG4_Gtract.bed > promoter_stableG4_Gtract.bed
awk -v OFS="\t" '{print $1,$2-1,$3-1}' ../data/promoter_unstable_Gtracts.bed > promoter_unstable_Gtracts.bed
awk -v OFS="\t" '{print $1,$2-1,$3-1}' ../data/active_promoter_pG4_Gtract.bed > active_promoter_pG4_Gtract.bed
awk -v OFS="\t" '{print $1,$2-1,$3-1}' ../data/active_promoter_stableG4_Gtract.bed > active_promoter_stableG4_Gtract.bed
awk -v OFS="\t" '{print $1,$2-1,$3-1}' ../data/active_promoter_unstable_Gtracts.bed > active_promoter_unstable_Gtracts.bed
mkdir pG4_popGenome
bcftools filter -R promoter_pG4.bed ../data/promoter.kg.norm_PASS_SNP.vcf.gz -O v -o pG4_popGenome/promoter_pG4.kg.norm_PASS_SNP.vcf
bcftools filter -R promoter_pG4_Stable.bed ../data/promoter.kg.norm_PASS_SNP.vcf.gz -O v -o pG4_popGenome/promoter_pG4_Stable.kg.norm_PASS_SNP.vcf
bcftools filter -R promoter_pG4_unStable.bed ../data/promoter.kg.norm_PASS_SNP.vcf.gz -O v -o pG4_popGenome/promoter_pG4_unStable.kg.norm_PASS_SNP.vcf
mkdir pG4_Gtract_popGenome
bcftools filter -R promoter_pG4_Gtract.bed ../data/promoter.kg.norm_PASS_SNP.vcf.gz -O v -o pG4_Gtract_popGenome/promoter_pG4_Gtract.kg.norm_PASS_SNP.vcf
bcftools filter -R promoter_stableG4_Gtract.bed ../data/promoter.kg.norm_PASS_SNP.vcf.gz -O v -o pG4_Gtract_popGenome/promoter_stableG4_Gtract.kg.norm_PASS_SNP.vcf
bcftools filter -R promoter_unstable_Gtracts.bed ../data/promoter.kg.norm_PASS_SNP.vcf.gz -O v -o pG4_Gtract_popGenome/promoter_unstable_Gtracts.kg.norm_PASS_SNP.vcf

mkdir eG4_popGenome
bcftools filter -R active_promoter_pG4.bed ../data/active_promoter.kg.norm_PASS_SNP.vcf.gz -O v -o eG4_popGenome/active_promoter_pG4.kg.norm_PASS_SNP.vcf
bcftools filter -R active_promoter_pG4_Stable.bed ../data/active_promoter.kg.norm_PASS_SNP.vcf.gz -O v -o eG4_popGenome/active_promoter_pG4_Stable.kg.norm_PASS_SNP.vcf
bcftools filter -R active_promoter_pG4_unStable.bed ../data/active_promoter.kg.norm_PASS_SNP.vcf.gz -O v -o eG4_popGenome/active_promoter_pG4_unStable.kg.norm_PASS_SNP.vcf
mkdir eG4_Gtract_popGenome
bcftools filter -R active_promoter_pG4_Gtract.bed ../data/active_promoter.kg.norm_PASS_SNP.vcf.gz -O v -o eG4_Gtract_popGenome/active_promoter_pG4_Gtract.kg.norm_PASS_SNP.vcf
bcftools filter -R active_promoter_stableG4_Gtract.bed ../data/active_promoter.kg.norm_PASS_SNP.vcf.gz -O v -o eG4_Gtract_popGenome/active_promoter_stableG4_Gtract.kg.norm_PASS_SNP.vcf
bcftools filter -R active_promoter_unstable_Gtracts.bed ../data/active_promoter.kg.norm_PASS_SNP.vcf.gz -O v -o eG4_Gtract_popGenome/active_promoter_unstable_Gtracts.kg.norm_PASS_SNP.vcf

grep -v "chrX" pG4_popGenome/promoter_pG4.kg.norm_PASS_SNP.vcf|grep -v "chrY" |grep -v "chrM" > pG4_popGenome/autosome.promoter_pG4.kg.norm_PASS_SNP.vcf
grep -v "chrX" pG4_popGenome/promoter_pG4_Stable.kg.norm_PASS_SNP.vcf|grep -v "chrY" |grep -v "chrM" > pG4_popGenome/autosome.promoter_pG4_Stable.kg.norm_PASS_SNP.vcf
grep -v "chrX" pG4_popGenome/promoter_pG4_unStable.kg.norm_PASS_SNP.vcf|grep -v "chrY" |grep -v "chrM" > pG4_popGenome/autosome.promoter_pG4_unStable.kg.norm_PASS_SNP.vcf

grep -v "chrX" pG4_Gtract_popGenome/promoter_pG4_Gtract.kg.norm_PASS_SNP.vcf |grep -v "chrY" |grep -v "chrM" > pG4_Gtract_popGenome/autosome.promoter_pG4_Gtract.kg.norm_PASS_SNP.vcf
grep -v "chrX" pG4_Gtract_popGenome/promoter_stableG4_Gtract.kg.norm_PASS_SNP.vcf |grep -v "chrY" |grep -v "chrM" > pG4_Gtract_popGenome/autosome.promoter_stableG4_Gtract.kg.norm_PASS_SNP.vcf
grep -v "chrX" pG4_Gtract_popGenome/promoter_unstable_Gtracts.kg.norm_PASS_SNP.vcf |grep -v "chrY" |grep -v "chrM" > pG4_Gtract_popGenome/autosome.promoter_unstable_Gtracts.kg.norm_PASS_SNP.vcf

grep -v "chrX" eG4_popGenome/active_promoter_pG4.kg.norm_PASS_SNP.vcf |grep -v "chrY" |grep -v "chrM" > eG4_popGenome/autosome.active_promoter_pG4.kg.norm_PASS_SNP.vcf
grep -v "chrX" eG4_popGenome/active_promoter_pG4_Stable.kg.norm_PASS_SNP.vcf |grep -v "chrY" |grep -v "chrM" > eG4_popGenome/autosome.active_promoter_pG4_Stable.kg.norm_PASS_SNP.vcf
grep -v "chrX" eG4_popGenome/active_promoter_pG4_unStable.kg.norm_PASS_SNP.vcf |grep -v "chrY" |grep -v "chrM" > eG4_popGenome/autosome.active_promoter_pG4_unStable.kg.norm_PASS_SNP.vcf

grep -v "chrX" eG4_Gtract_popGenome/active_promoter_pG4_Gtract.kg.norm_PASS_SNP.vcf |grep -v "chrY"|grep -v "chrM" > eG4_Gtract_popGenome/autosome.active_promoter_pG4_Gtract.kg.norm_PASS_SNP.vcf
grep -v "chrX" eG4_Gtract_popGenome/active_promoter_stableG4_Gtract.kg.norm_PASS_SNP.vcf |grep -v "chrY"|grep -v "chrM" > eG4_Gtract_popGenome/autosome.active_promoter_stableG4_Gtract.kg.norm_PASS_SNP.vcf
grep -v "chrX" eG4_Gtract_popGenome/active_promoter_unstable_Gtracts.kg.norm_PASS_SNP.vcf |grep -v "chrY"|grep -v "chrM" > eG4_Gtract_popGenome/autosome.active_promoter_unstable_Gtracts.kg.norm_PASS_SNP.vcf


In [ ]:
## work.R
library("PopGenome")
args <- commandArgs(T)
g4_type = args[1]
vcfpath = args[2]
setwd("/disk2/G4_MS/frequency/evolution/analysis")

#pG4 <- readData("pG4_Gtract_popGenome/promoter_stableG4_Gtract", format="VCF")
pG4 <- readData(vcfpath, format="VCF")

neut <- neutrality.stats(pG4)
df1 <- data.frame(Tajima.D=c(neut@Tajima.D),Fu.Li.D=c(neut@Fu.Li.D), Fu.Li.F=c(neut@Fu.Li.F))
df1$groups = gsub("_", " ", g4_type)
row.names(df1) = "Merge"
population_select <- function(population){
    new_population <- c()
    for (i in population){
        new_population <- c(new_population, i, paste(i,".2",sep="") )
    }
    return(new_population)
}

AFR <- read.table("../data/population_AFR.csv",header=F)
EUR <- read.table("../data/population_EUR.csv",header=F)
AMR <- read.table("../data/population_AMR.csv",header=F)
EAS <- read.table("../data/population_EAS.csv",header=F)
SAS <- read.table("../data/population_SAS.csv",header=F)

AFR.2 = population_select(AFR)
AMR.2 = population_select(AMR)
SAS.2 = population_select(SAS)
EAS.2 = population_select(EAS)
EUR.2 = population_select(EUR)

neut <- set.populations(neut, list(EUR=EUR.2, AFR=AFR.2,AMR=AMR.2,SAS=SAS.2,EAS=EAS.2))
neut <- neutrality.stats(neut)
df <- data.frame(Tajima.D=c(neut@Tajima.D), Fu.Li.D=c(neut@Fu.Li.D), Fu.Li.F=c(neut@Fu.Li.F))
df$groups = gsub("_"," ", g4_type)
row.names(df) <- c("EUR", "AFR", "AMR", "SAS","EAS")
df <- rbind(df1,df)
write.table(df, paste(g4_type,".evolution.tsv",sep=""),sep="\t")

In [ ]:
%%bash
cd /disk2/G4_MS/frequency/evolution/analysis
Rscript work.R active_promoter_pG4_Gtract eG4_Gtract_popGenome/active_promoter_pG4_Gtract
Rscript work.R active_promoter_stableG4_Gtract eG4_Gtract_popGenome/active_promoter_stableG4_Gtract
Rscript work.R active_promoter_unstable_Gtracts eG4_Gtract_popGenome/active_promoter_unstable_Gtracts
Rscript work.R active_promoter_pG4 eG4_popGenome/active_promoter_pG4
Rscript work.R active_promoter_pG4_Stable eG4_popGenome/active_promoter_pG4_Stable
Rscript work.R active_promoter_pG4_unStable eG4_popGenome/active_promoter_pG4_unStable

Rscript work.R promoter_pG4_Gtract pG4_Gtract_popGenome/promoter_pG4_Gtract
Rscript work.R promoter_stableG4_Gtract pG4_Gtract_popGenome/promoter_stableG4_Gtract
Rscript work.R promoter_unstable_Gtracts pG4_Gtract_popGenome/promoter_unstable_Gtracts

Rscript work.R promoter_pG4 pG4_popGenome/promoter_pG4
Rscript work.R promoter_pG4_Stable pG4_popGenome/promoter_pG4_Stable
Rscript work.R promoter_pG4_unStable pG4_popGenome/promoter_pG4_unStable


In [ ]:
# result filter and export
import os
import pandas as pd
import numpy as np

def file_get(path):
    file_list = []
    files = os.listdir(path)
    for f in files:
        if f.endswith("evolution.tsv"):
            file_list.append(os.path.join(path,f))
    return file_list
            
            
wd = "/disk2/G4_MS/frequency/evolution/analysis"
df_list = []
for f in file_get(wd):
    df_list.append(pd.read_table(f))
df = pd.concat(df_list)
df = df.iloc[:,[3,0,1,2]]
df = df.round(3)

with pd.ExcelWriter(os.path.join(wd,"Evolution.xlsx")) as writer:
    df[df.index=="Merge"].sort_values("Tajima.D").to_excel(writer,sheet_name="1000g evolution", index=False)
    df[df.index=="EUR"].sort_values("Tajima.D").to_excel(writer,sheet_name="EUR", index=False)
    df[df.index=="EAS"].sort_values("Tajima.D").to_excel(writer,sheet_name="EAS", index=False)
    df[df.index=="AFR"].sort_values("Tajima.D").to_excel(writer,sheet_name="AFR", index=False)
    df[df.index=="AMR"].sort_values("Tajima.D").to_excel(writer,sheet_name="AMR", index=False)
    df[df.index=="SAS"].sort_values("Tajima.D").to_excel(writer,sheet_name="SAS", index=False)